# Project Extensions
We choose to prune and quantize the baselines in order to obtain a smaller model by dropping performance as low as possible

In [ ]:
%cd /content
!git clone https://github.com/m-depascale/AnomalySegmentation_AML

/content
Cloning into 'AnomalySegmentation_AML'...
remote: Enumerating objects: 438, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 438 (delta 69), reused 93 (delta 55), pack-reused 320
Receiving objects: 100% (438/438), 363.44 MiB | 23.17 MiB/s, done.
Resolving deltas: 100% (233/233), done.
Updating files: 100% (82/82), done.


In [ ]:
!pip install ood-metrics
!pip install visdom
!pip install numpy
!pip install matplotlib
!pip install Pillow
!pip install torchvision
!python -m pip install cityscapesscripts
!python -m pip install cityscapesscripts[gui]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408194 sha256=0075e8887bc37aecab6778f19230c84b8340dd0347fd3ce12f6665e4824be77b
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.3/473.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.7 MB/s eta 0:00:00
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=9c8350afc475996e18aaaad183fb84ce5d9f43ae7df9810b9334fca50b393fb6
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc

In [ ]:
#As shows in the README here: https://github.com/mcordts/cityscapesScripts
from cityscapesscripts.download import downloader
print(dir(downloader))
import os

# Esegui il login
session = downloader.login()
names = downloader.list_available_packages(session=session)
print(names)


destination_path = "/content/Datasets/Cityscapes"
os.makedirs(destination_path, exist_ok=True)

downloader.download_packages(session=session, package_names=['leftImg8bit_trainvaltest.zip', 'gtFine_trainvaltest.zip'], destination_path=destination_path)

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'absolute_import', 'appdirs', 'argparse', 'division', 'download_packages', 'get_available_packages', 'getpass', 'hashlib', 'input', 'json', 'list_available_packages', 'login', 'main', 'os', 'parse_arguments', 'print_function', 'requests', 'shutil', 'stat', 'unicode_literals']
Cityscapes username or email address: mdepa
Cityscapes password: ··········
Store credentials unencrypted in '/root/.local/share/cityscapesscripts/credentials.json' [y/N]: 
The following packages are available for download.
Please refer to https://www.cityscapes-dataset.com/downloads/ for additional packages and instructions on properly citing third party packages.
 gtFine_trainvaltest.zip -> 241MB
 gtCoarse.zip -> 1.3GB
 leftImg8bit_trainvaltest.zip -> 11GB
 leftImg8bit_trainextra.zip -> 44GB
 rightImg8bit_trainvaltest.zip -> 11GB
 rightImg8bit_trainextra.zip -> 44GB
 disparity_trainvaltest.zip -> 3.5GB
 dis

In [ ]:
!unzip /content/Datasets/Cityscapes/leftImg8bit_trainvaltest.zip -d /content/Datasets/Cityscapes/
!unzip /content/Datasets/Cityscapes/gtFine_trainvaltest.zip -d /content/Datasets/Cityscapes/

Streaming output truncated to the last 5000 lines.
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000117_000019_gtFine_color.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000114_000019_gtFine_color.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000434_000019_gtFine_labelIds.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000420_000019_gtFine_color.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000483_000019_gtFine_instanceIds.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000420_000019_gtFine_instanceIds.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000254_000019_gtFine_color.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000490_000019_gtFine_color.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000448_000019_gtFine_polygons.json  
  inflating: /content

In [ ]:
os.environ['CITYSCAPES_DATASET'] = '/content/Datasets/Cityscapes'
from cityscapesscripts.preparation.createTrainIdLabelImgs import main as convertor
convertor()

Processing 5000 annotation files
Progress: 100.0 % 

In [ ]:
%cd /content/AnomalySegmentation_AML/trained_models/
# Extract the pieces
!unrar x /content/AnomalySegmentation_AML/trained_models/bisenet_finetuned_final.part1.rar
!unrar x /content/AnomalySegmentation_AML/trained_models/bisenet_finetuned_final.part2.rar
!unrar x /content/AnomalySegmentation_AML/trained_models/bisenet_finetuned_final.part3.rar

#Select "All" when colab asks you something.
#Then you will find bisenet_finetuned.pth on /content/bisenet_finetuned.pth

## Pruning
We decided to apply **Structured Pruning** to ERFNet because it's structure is simpler so we can manage that more easily. In particular, Channel-based pruning aims to remove some filters, so we end up with a smaller model.

Pytorch does not support structured pruning that much, it is only possible to use prune.ln_structured method to obtain a mask for the weights of a certain dimension, but we have to modify more dimensions. Also, it does not return a "smaller" module, so the inference time and model size remains the same (actually, they gets a bit higher due to the masking process).

Instead, we looked at the architecture ad decided to prune first the decoder of the network. We then froze the other layers and trained the decoder part only, in order to let the network compensate for the changing architecture.

After 20 epochs of training with initial LR=0.0002, we got that mIoU=68.50% (consider the full network had 72.03% when trained on 20 classes), which is an interesting results.

We proceeded by now changing the encoder part of the network. Here the architecture had more layers, so we decided to maintain half of them and then using Downsampler and Upsampler blocks with the same logic of the author to pass from 128 channels to 32 channels and then returning on 128 channels for not changing the decoder again.

Again, we froze the decoder and trained only the encoder part with the same settings as the decoder, obtaining mIoU=63.65%.

Finally, we fine-tuned all the network for a short period (10 epochs) and achieved mIoU on val set of 65.44%

The proposed architecture change for the ERFNet is the following:

In [ ]:
# ERFNet full model definition for Pytorch
# Sept 2017
# Eduardo Romera
#######################

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

class DownsamplerBlock (nn.Module):
    def __init__(self, ninput, noutput):
        super().__init__()

        self.conv = nn.Conv2d(ninput, noutput-ninput, (3, 3), stride=2, padding=1, bias=True)
        self.pool = nn.MaxPool2d(2, stride=2)
        self.bn = nn.BatchNorm2d(noutput, eps=1e-3)

    def forward(self, input):
        output = torch.cat([self.conv(input), self.pool(input)], 1)
        output = self.bn(output)
        return F.relu(output)


class non_bottleneck_1d (nn.Module):
    def __init__(self, chann, dropprob, dilated, cout=0):
        super().__init__()

        self.conv3x1_1 = nn.Conv2d(chann, chann, (3, 1), stride=1, padding=(1,0), bias=True)

        self.conv1x3_1 = nn.Conv2d(chann, chann, (1,3), stride=1, padding=(0,1), bias=True)

        self.bn1 = nn.BatchNorm2d(chann, eps=1e-03)

        self.conv3x1_2 = nn.Conv2d(chann, chann, (3, 1), stride=1, padding=(1*dilated,0), bias=True, dilation = (dilated,1))

        self.conv1x3_2 = nn.Conv2d(chann, chann, (1,3), stride=1, padding=(0,1*dilated), bias=True, dilation = (1, dilated))

        self.bn2 = nn.BatchNorm2d(chann, eps=1e-03)

        self.dropout = nn.Dropout2d(dropprob)


    def forward(self, input):

        output = self.conv3x1_1(input)
        output = F.relu(output)
        output = self.conv1x3_1(output)
        output = self.bn1(output)
        output = F.relu(output)

        output = self.conv3x1_2(output)
        output = F.relu(output)
        output = self.conv1x3_2(output)
        output = self.bn2(output)

        if (self.dropout.p != 0):
            output = self.dropout(output)

        return F.relu(output+input)    #+input = identity (residual connection)


class Encoder(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.initial_block = DownsamplerBlock(3,16)

        self.layers = nn.ModuleList()

        self.layers.append(DownsamplerBlock(16,64))

        for x in range(0, 5):    #5 times
           self.layers.append(non_bottleneck_1d(64, 0.03, 1))

        self.layers.append(DownsamplerBlock(64,128))

        #for x in range(0, 2):    #2 times
        self.layers.append(non_bottleneck_1d(128, 0.3, 2))
        self.layers.append(non_bottleneck_1d(128, 0.3, 4))
        self.layers.append(non_bottleneck_1d(128, 0.3, 8))
        self.layers.append(non_bottleneck_1d(128, 0.3, 16))

        self.layers.append(UpsamplerBlock(128,32))

        self.layers.append(non_bottleneck_1d(32, 0.15, 1))
        self.layers.append(non_bottleneck_1d(32, 0.15, 2))
        self.layers.append(non_bottleneck_1d(32, 0.15, 4))
        self.layers.append(non_bottleneck_1d(32, 0.15, 8))

        self.layers.append(DownsamplerBlock(32,128))
        #Only in encoder mode:
        self.output_conv = nn.Conv2d(128, num_classes, 1, stride=1, padding=0, bias=True)

    def forward(self, input, predict=False):
        output = self.initial_block(input)

        for layer in self.layers:
            output = layer(output)

        if predict:
            output = self.output_conv(output)

        return output


class UpsamplerBlock (nn.Module):
    def __init__(self, ninput, noutput):
        super().__init__()
        self.conv = nn.ConvTranspose2d(ninput, noutput, 3, stride=2, padding=1, output_padding=1, bias=True)
        self.bn = nn.BatchNorm2d(noutput, eps=1e-3)

    def forward(self, input):
        output = self.conv(input)
        output = self.bn(output)
        return F.relu(output)

class Decoder (nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.layers = nn.ModuleList()

        self.layers.append(UpsamplerBlock(128,32))
        self.layers.append(non_bottleneck_1d(32, 0, 1))
        self.layers.append(non_bottleneck_1d(32, 0, 1))

        self.layers.append(UpsamplerBlock(32,16))
        self.layers.append(non_bottleneck_1d(16, 0, 1))
        self.layers.append(non_bottleneck_1d(16, 0, 1))

        self.output_conv = nn.ConvTranspose2d( 16, num_classes, 2, stride=2, padding=0, output_padding=0, bias=True)

    def forward(self, input):
        output = input

        for layer in self.layers:
            output = layer(output)

        output = self.output_conv(output)

        return output

#ERFNet
class ERFNET_PRUNEDV2(nn.Module):
    def __init__(self, num_classes, encoder=None):  #use encoder to pass pretrained encoder
        super().__init__()

        if (encoder == None):
            self.encoder = Encoder(num_classes)
        else:
            self.encoder = encoder
        self.decoder = Decoder(num_classes)

    def forward(self, input, only_encode=False):
        if only_encode:
            return self.encoder.forward(input, predict=True)
        else:
            output = self.encoder(input)    #predict=False by default
            return self.decoder.forward(output)

Let's now check the differences in model size, parameters and inference time (on GPU)

In [ ]:
%cd /content/AnomalySegmentation_AML/eval/

/content/AnomalySegmentation_AML/eval


In [ ]:
from erfnet import ERFNet
from torchsummary import summary

erfnet_full = ERFNet(20)
erfnet_pruned = ERFNET_PRUNEDV2(20)

erfnet_full.cuda()
erfnet_pruned.cuda()

input = (3, 512, 1024)

In [ ]:
erfnet_pruned

ERFNET_PRUNEDV2(
  (encoder): Encoder(
    (initial_block): DownsamplerBlock(
      (conv): Conv2d(3, 13, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (layers): ModuleList(
      (0): DownsamplerBlock(
        (conv): Conv2d(16, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1-5): 5 x non_bottleneck_1d(
        (conv3x1_1): Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (conv1x3_1): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (bn1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (conv3x1_2): 

In [ ]:
summary(erfnet_full, input)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 13, 256, 512]             364
         MaxPool2d-2          [-1, 3, 256, 512]               0
       BatchNorm2d-3         [-1, 16, 256, 512]              32
  DownsamplerBlock-4         [-1, 16, 256, 512]               0
            Conv2d-5         [-1, 48, 128, 256]           6,960
         MaxPool2d-6         [-1, 16, 128, 256]               0
       BatchNorm2d-7         [-1, 64, 128, 256]             128
  DownsamplerBlock-8         [-1, 64, 128, 256]               0
            Conv2d-9         [-1, 64, 128, 256]          12,352
           Conv2d-10         [-1, 64, 128, 256]          12,352
      BatchNorm2d-11         [-1, 64, 128, 256]             128
           Conv2d-12         [-1, 64, 128, 256]          12,352
           Conv2d-13         [-1, 64, 128, 256]          12,352
      BatchNorm2d-14         [-1, 64, 1

In [ ]:
summary(erfnet_pruned, input)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 13, 256, 512]             364
         MaxPool2d-2          [-1, 3, 256, 512]               0
       BatchNorm2d-3         [-1, 16, 256, 512]              32
  DownsamplerBlock-4         [-1, 16, 256, 512]               0
            Conv2d-5         [-1, 48, 128, 256]           6,960
         MaxPool2d-6         [-1, 16, 128, 256]               0
       BatchNorm2d-7         [-1, 64, 128, 256]             128
  DownsamplerBlock-8         [-1, 64, 128, 256]               0
            Conv2d-9         [-1, 64, 128, 256]          12,352
           Conv2d-10         [-1, 64, 128, 256]          12,352
      BatchNorm2d-11         [-1, 64, 128, 256]             128
           Conv2d-12         [-1, 64, 128, 256]          12,352
           Conv2d-13         [-1, 64, 128, 256]          12,352
      BatchNorm2d-14         [-1, 64, 1

### Results of pruning ERFNET

We passed from 2,064,256 trainable parameters to 1,273,056, which constitutes a 1.62 reduction factor (full / pruned).

For the model params size, we passed from 7.87 MB to 4.86 MB, with a 1.62 recution factor.

Let's check the inference time with a simple script.

In [ ]:
import torch
import time
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import FakeData
import numpy as np

def inference_time(model, dataloader, device):
    model.eval()
    total_time = 0.0
    num_samples = 0

    with torch.no_grad():
        for inputs in dataloader:
            image = inputs[0].cuda()

            start_time = time.time()
            _ = model(image)  # Perform inference, adjust as needed based on your model's forward pass
            end_time = time.time()

            total_time += end_time - start_time
            num_samples += 1

    avg_time = total_time / num_samples
    return avg_time

# Set a random seed for reproducibility
torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a data loader with random images
transform = transforms.Compose([transforms.ToTensor()])
fake_dataset = FakeData(transform=transform)
test_loader = DataLoader(fake_dataset, batch_size=1, shuffle=True, num_workers=2)

# Number of runs for obtaining average and standard deviation
num_runs = 100
inference_times = []

for run in range(num_runs):
    # Measure inference time for model1
    time_model1 = inference_time(erfnet_full, test_loader, device)
    inference_times.append(time_model1)
    print(f"Run {run + 1}: Inference time for full model: {time_model1:.6f} seconds per sample")

# Calculate average and standard deviation
avg_time_model1 = np.mean(inference_times)
std_dev_model1 = np.std(inference_times)

print(f"\nAverage inference time for full model: {avg_time_model1:.6f} seconds per sample")
print(f"Standard deviation for full model: {std_dev_model1:.6f} seconds per sample\n")

# Repeat the process for model2
inference_times = []

for run in range(num_runs):
    time_model2 = inference_time(erfnet_pruned, test_loader, device)
    inference_times.append(time_model2)
    print(f"Run {run + 1}: Inference time for pruned model: {time_model2:.6f} seconds per sample")

# Calculate average and standard deviation
avg_time_model2 = np.mean(inference_times)
std_dev_model2 = np.std(inference_times)

print(f"\nAverage inference time for pruned model: {avg_time_model2:.6f} seconds per sample")
print(f"Standard deviation for pruned model: {std_dev_model2:.6f} seconds per sample\n")

# Compare the results
if avg_time_model1 < avg_time_model2:
    print("Full ERFNet has faster inference time.")
elif avg_time_model1 > avg_time_model2:
    print("Pruned ERFNet has faster inference time.")
else:
    print("Both models have similar average inference times.")


Run 1: Inference time for full model: 0.012339 seconds per sample
Run 2: Inference time for full model: 0.012476 seconds per sample
Run 3: Inference time for full model: 0.013685 seconds per sample
Run 4: Inference time for full model: 0.013120 seconds per sample
Run 5: Inference time for full model: 0.013023 seconds per sample
Run 6: Inference time for full model: 0.014038 seconds per sample
Run 7: Inference time for full model: 0.013581 seconds per sample
Run 8: Inference time for full model: 0.013594 seconds per sample
Run 9: Inference time for full model: 0.012762 seconds per sample
Run 10: Inference time for full model: 0.013154 seconds per sample
Run 11: Inference time for full model: 0.013574 seconds per sample
Run 12: Inference time for full model: 0.013223 seconds per sample
Run 13: Inference time for full model: 0.012496 seconds per sample
Run 14: Inference time for full model: 0.012288 seconds per sample
Run 15: Inference time for full model: 0.012163 seconds per sample
Run 

These are the results for the pruned model on a powerful GPU, but what about we have to run it on less powerful devices that might not have GPU and needs to run in on CPU?

We want to squeeze the performance as maximum we can, so we use quantization to use less bits for the parameters in the model, and also the backend used allows for some GEMM optimizations. In this case, with colab, we can't test cudnn backend for quantized model, so we are limited on CPU only, but let's see that after.

In [ ]:
qconfig = get_default_qconfig('x86')
qconfig = qconfig._replace(
    activation=MovingAverageMinMaxObserver.with_args(dtype=torch.qint8, qscheme=torch.per_tensor_symmetric),
    weight=MovingAverageMinMaxObserver.with_args(dtype=torch.qint8, qscheme=torch.per_tensor_symmetric))
qconfig_mapping = QConfigMapping().set_global(qconfig)

In [ ]:
%cd /content/AnomalySegmentation_AML/eval

/content/AnomalySegmentation_AML/eval


In [ ]:
torch.backends.quantized.engine = 'x86'

## Quantization
There are differents method for quantizating a model, we tried to use Post Training Static quantization, but the results are not good at all. Also, it is not possible to train (fine tune) a quantized model with this approach, so it is not optimal.


In [ ]:
import os
import random
from PIL import Image
import numpy as np
import copy
import glob
from torchsummary import summary
import torch
import torch.fx
import torch.nn as nn
import torch.optim as optim
import torch.quantization
from torch.utils.data import DataLoader
from torchvision.transforms import Resize, transforms
from torch.ao.quantization import get_default_qconfig, get_default_qconfig_mapping, get_default_qat_qconfig_mapping, prepare_qat, convert
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx, prepare_qat_fx
from torch.ao.quantization import QConfigMapping
from torch.quantization.observer import MovingAverageMinMaxObserver
from torch.ao.quantization.observer import FixedQParamsObserver
import torch.nn.utils.prune as prune
import torch.nn.functional as F

In [ ]:
# we need some calibration data, so we take them randomly from the test set
main_folder = '/content/Datasets/Cityscapes/leftImg8bit/train/'
paths = []
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)

    # Check if the item is a directory (subfolder)
    if os.path.isdir(subfolder_path):
        print(f"Selecting 10% random images from subfolder: {subfolder}")

        # Get a list of all image files in the subfolder
        image_files = [filename for filename in os.listdir(subfolder_path) if filename.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Select num_random_images random images from the list
        selected_images = random.sample(image_files, round(len(image_files)*0.1))

        # Process each selected image
        for filename in selected_images:
            file_path = os.path.join(subfolder_path, filename)

            # Append the path to the list
            paths.append(file_path)

calibration_data = []
for path in paths:
  images = torch.from_numpy(np.array(Image.open(path).convert('RGB'))).unsqueeze(0).float()
  images = images.permute(0,3,1,2)
  images = Resize((512,1024), Image.BILINEAR)(images) #resize only when dealing with bisenet
  calibration_data.append(images)

Selecting 10% random images from subfolder: cologne
Selecting 10% random images from subfolder: erfurt
Selecting 10% random images from subfolder: hanover
Selecting 10% random images from subfolder: zurich
Selecting 10% random images from subfolder: weimar
Selecting 10% random images from subfolder: strasbourg
Selecting 10% random images from subfolder: stuttgart
Selecting 10% random images from subfolder: bremen
Selecting 10% random images from subfolder: ulm
Selecting 10% random images from subfolder: aachen
Selecting 10% random images from subfolder: dusseldorf
Selecting 10% random images from subfolder: bochum
Selecting 10% random images from subfolder: monchengladbach
Selecting 10% random images from subfolder: darmstadt
Selecting 10% random images from subfolder: jena
Selecting 10% random images from subfolder: hamburg
Selecting 10% random images from subfolder: krefeld
Selecting 10% random images from subfolder: tubingen


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
%cd /content/AnomalySegmentation_AML/eval

/content/AnomalySegmentation_AML/eval


In [ ]:
import torch
from bisenetv1 import BiSeNetV1

from torch.ao.quantization import (
  get_default_qconfig_mapping,
  get_default_qat_qconfig_mapping,
  QConfigMapping,
)
import torch.ao.quantization.quantize_fx as quantize_fx
import copy

In [ ]:
bisenet_fp32 = BiSeNetV1(20)
bisenet_fp32.load_state_dict(torch.load('/content/AnomalySegmentation_AML/trained_models/bisenet_finetuned_final.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:

bisenet_to_quantize = copy.deepcopy(bisenet_fp32)
qconfig_mapping = get_default_qconfig_mapping("x86")
bisenet_to_quantize.eval()
# prepare
bisenet_prepared = quantize_fx.prepare_fx(bisenet_to_quantize, qconfig_mapping, calibration_data)
with torch.no_grad():
  for idx, image in enumerate(calibration_data):
    print(idx)
    bisenet_prepared(image)
# quantize
bisenet_ptsq = quantize_fx.convert_fx(bisenet_prepared)

In [ ]:
torch.save(bisenet_ptsq.state_dict(), '/content/AnomalySegmentation_AML/trained_models/bisenet_ptsq.pth')
print('Bisenet (PTSQ) Size (MB):', os.path.getsize("/content/AnomalySegmentation_AML/trained_models/bisenet_ptsq.pth")/1e6)

In [ ]:
# @title
# Code with dataset loader for VOC12 and Cityscapes (adapted from bodokaiser/piwise code)
# Sept 2017
# Eduardo Romera
#######################

import numpy as np
import os

from PIL import Image

from torch.utils.data import Dataset

EXTENSIONS = ['.jpg', '.png']

def load_image(file):
    return Image.open(file)

def is_image(filename):
    return any(filename.endswith(ext) for ext in EXTENSIONS)

def is_label(filename):
    return filename.endswith("_labelTrainIds.png")

def image_path(root, basename, extension):
    return os.path.join(root, f'{basename}{extension}')

def image_path_city(root, name):
    return os.path.join(root, f'{name}')

def image_basename(filename):
    return os.path.basename(os.path.splitext(filename)[0])

class VOC12(Dataset):

    def __init__(self, root, input_transform=None, target_transform=None):
        self.images_root = os.path.join(root, 'images')
        self.labels_root = os.path.join(root, 'labels')

        self.filenames = [image_basename(f)
            for f in os.listdir(self.labels_root) if is_image(f)]
        self.filenames.sort()

        self.input_transform = input_transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        filename = self.filenames[index]

        with open(image_path(self.images_root, filename, '.jpg'), 'rb') as f:
            image = load_image(f).convert('RGB')
        with open(image_path(self.labels_root, filename, '.png'), 'rb') as f:
            label = load_image(f).convert('P')

        if self.input_transform is not None:
            image = self.input_transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)

        return image, label

    def __len__(self):
        return len(self.filenames)


class cityscapes(Dataset):

    def __init__(self, root, input_transform=None, target_transform=None, subset='val'):

        self.images_root = os.path.join(root, 'leftImg8bit/' + subset)
        self.labels_root = os.path.join(root, 'gtFine/' + subset)
        print(self.images_root, self.labels_root)
        self.filenames = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(self.images_root)) for f in fn if is_image(f)]
        self.filenames.sort()

        self.filenamesGt = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(self.labels_root)) for f in fn if is_label(f)]
        self.filenamesGt.sort()

        self.input_transform = input_transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        filename = self.filenames[index]
        filenameGt = self.filenamesGt[index]

        #print(filename)

        with open(image_path_city(self.images_root, filename), 'rb') as f:
            image = load_image(f).convert('RGB')
        with open(image_path_city(self.labels_root, filenameGt), 'rb') as f:
            label = load_image(f).convert('P')

        if self.input_transform is not None:
            image = self.input_transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)

        return image, label, filename, filenameGt

    def __len__(self):
        return len(self.filenames)



In [ ]:
# @title
import numpy as np
import torch
import torch.nn.functional as F
import os
import importlib
import time

from PIL import Image
from argparse import ArgumentParser

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize
from torchvision.transforms import ToTensor, ToPILImage

from transform import Relabel, ToLabel, Colorize
from iouEval import iouEval, getColorEntry

NUM_CHANNELS = 3
NUM_CLASSES = 20

image_transform = ToPILImage()
input_transform_cityscapes = Compose([
    Resize(512, Image.BILINEAR),
    ToTensor(),
])
target_transform_cityscapes = Compose([
    Resize(512, Image.NEAREST),
    ToLabel(),
    Relabel(255, 19),   #ignore label to 19
])

loader = DataLoader(cityscapes('/content/Datasets/Cityscapes/', input_transform=input_transform_cityscapes, target_transform=target_transform_cityscapes, subset='val'), num_workers=2, batch_size=1, shuffle=False)

iouEvalVal = iouEval(NUM_CLASSES)

start = time.time()
bisenet_ptsq.to(torch.device('cpu'))
bisenet_ptsq.eval()
bisenet_ptsq.aux_mode='eval'


i=0
for step, (images, labels, filename, filenameGt) in enumerate(loader):

    inputs = Variable(images)
    #inputs_8bit = torch.quantize_per_tensor(inputs, 0.1, 10, torch.quint8)
    with torch.no_grad():
        outputs = bisenet_ptsq(inputs)
        #fp32_outputs = fp32_model(inputs)
    iouEvalVal.addBatch(outputs[1].max(1)[1].unsqueeze(1).data, labels)

    filenameSave = filename[0].split("leftImg8bit/")[1]

    print (step, filenameSave)
    i +=1
iouVal, iou_classes = iouEvalVal.getIoU()

iou_classes_str = []
for i in range(iou_classes.size(0)):
    iouStr = getColorEntry(iou_classes[i])+'{:0.2f}'.format(iou_classes[i]*100) + '\033[0m'
    iou_classes_str.append(iouStr)

print("---------------------------------------")
print("Took ", time.time()-start, "seconds")
print("=======================================")
#print("TOTAL IOU: ", iou * 100, "%")
print("Per-Class IoU:")
print(iou_classes_str[0], "Road")
print(iou_classes_str[1], "sidewalk")
print(iou_classes_str[2], "building")
print(iou_classes_str[3], "wall")
print(iou_classes_str[4], "fence")
print(iou_classes_str[5], "pole")
print(iou_classes_str[6], "traffic light")
print(iou_classes_str[7], "traffic sign")
print(iou_classes_str[8], "vegetation")
print(iou_classes_str[9], "terrain")
print(iou_classes_str[10], "sky")
print(iou_classes_str[11], "person")
print(iou_classes_str[12], "rider")
print(iou_classes_str[13], "car")
print(iou_classes_str[14], "truck")
print(iou_classes_str[15], "bus")
print(iou_classes_str[16], "train")
print(iou_classes_str[17], "motorcycle")
print(iou_classes_str[18], "bicycle")
print("=======================================")
iouStr = getColorEntry(iouVal)+'{:0.2f}'.format(iouVal*100) + '\033[0m'
print ("MEAN IoU: ", iouStr, "%")

/content/Datasets/Cityscapes/leftImg8bit/val /content/Datasets/Cityscapes/gtFine/val
0 val/frankfurt/frankfurt_000000_000294_leftImg8bit.png
1 val/frankfurt/frankfurt_000000_000576_leftImg8bit.png
2 val/frankfurt/frankfurt_000000_001016_leftImg8bit.png
3 val/frankfurt/frankfurt_000000_001236_leftImg8bit.png
4 val/frankfurt/frankfurt_000000_001751_leftImg8bit.png
5 val/frankfurt/frankfurt_000000_002196_leftImg8bit.png
6 val/frankfurt/frankfurt_000000_002963_leftImg8bit.png
7 val/frankfurt/frankfurt_000000_003025_leftImg8bit.png
8 val/frankfurt/frankfurt_000000_003357_leftImg8bit.png
9 val/frankfurt/frankfurt_000000_003920_leftImg8bit.png
10 val/frankfurt/frankfurt_000000_004617_leftImg8bit.png
11 val/frankfurt/frankfurt_000000_005543_leftImg8bit.png
12 val/frankfurt/frankfurt_000000_005898_leftImg8bit.png
13 val/frankfurt/frankfurt_000000_006589_leftImg8bit.png
14 val/frankfurt/frankfurt_000000_007365_leftImg8bit.png
15 val/frankfurt/frankfurt_000000_008206_leftImg8bit.png
16 val/frankf

Let's try changing the backend

In [ ]:
bisenet_to_quantize = copy.deepcopy(bisenet_fp32)
qconfig_mapping = get_default_qconfig_mapping("qnnpack")
bisenet_to_quantize.eval()
# prepare
bisenet_prepared = quantize_fx.prepare_fx(bisenet_to_quantize, qconfig_mapping, calibration_data)
with torch.no_grad():
  for idx, image in enumerate(calibration_data):
    print(idx)
    bisenet_prepared(image)
# quantize
bisenet_ptsq_qnnpack = quantize_fx.convert_fx(bisenet_prepared)

In [ ]:
# @title
import numpy as np
import torch
import torch.nn.functional as F
import os
import importlib
import time

from PIL import Image
from argparse import ArgumentParser

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize
from torchvision.transforms import ToTensor, ToPILImage

from transform import Relabel, ToLabel, Colorize
from iouEval import iouEval, getColorEntry

NUM_CHANNELS = 3
NUM_CLASSES = 20

image_transform = ToPILImage()
input_transform_cityscapes = Compose([
    Resize(512, Image.BILINEAR),
    ToTensor(),
])
target_transform_cityscapes = Compose([
    Resize(512, Image.NEAREST),
    ToLabel(),
    Relabel(255, 19),   #ignore label to 19
])

loader = DataLoader(cityscapes('/content/Datasets/Cityscapes/', input_transform=input_transform_cityscapes, target_transform=target_transform_cityscapes, subset='val'), num_workers=2, batch_size=1, shuffle=False)

iouEvalVal = iouEval(NUM_CLASSES)

start = time.time()
bisenet_ptsq_qnnpack.to(torch.device('cpu'))
bisenet_ptsq_qnnpack.eval()
bisenet_ptsq_qnnpack.aux_mode='eval'


i=0
for step, (images, labels, filename, filenameGt) in enumerate(loader):

    inputs = Variable(images)
    #inputs_8bit = torch.quantize_per_tensor(inputs, 0.1, 10, torch.quint8)
    with torch.no_grad():
        outputs = bisenet_ptsq_qnnpack(inputs)
        #fp32_outputs = fp32_model(inputs)
    iouEvalVal.addBatch(outputs[1].max(1)[1].unsqueeze(1).data, labels)

    filenameSave = filename[0].split("leftImg8bit/")[1]

    print (step, filenameSave)
    i +=1
iouVal, iou_classes = iouEvalVal.getIoU()

iou_classes_str = []
for i in range(iou_classes.size(0)):
    iouStr = getColorEntry(iou_classes[i])+'{:0.2f}'.format(iou_classes[i]*100) + '\033[0m'
    iou_classes_str.append(iouStr)

print("---------------------------------------")
print("Took ", time.time()-start, "seconds")
print("=======================================")
#print("TOTAL IOU: ", iou * 100, "%")
print("Per-Class IoU:")
print(iou_classes_str[0], "Road")
print(iou_classes_str[1], "sidewalk")
print(iou_classes_str[2], "building")
print(iou_classes_str[3], "wall")
print(iou_classes_str[4], "fence")
print(iou_classes_str[5], "pole")
print(iou_classes_str[6], "traffic light")
print(iou_classes_str[7], "traffic sign")
print(iou_classes_str[8], "vegetation")
print(iou_classes_str[9], "terrain")
print(iou_classes_str[10], "sky")
print(iou_classes_str[11], "person")
print(iou_classes_str[12], "rider")
print(iou_classes_str[13], "car")
print(iou_classes_str[14], "truck")
print(iou_classes_str[15], "bus")
print(iou_classes_str[16], "train")
print(iou_classes_str[17], "motorcycle")
print(iou_classes_str[18], "bicycle")
print("=======================================")
iouStr = getColorEntry(iouVal)+'{:0.2f}'.format(iouVal*100) + '\033[0m'
print ("MEAN IoU: ", iouStr, "%")

/content/Datasets/Cityscapes/leftImg8bit/val /content/Datasets/Cityscapes/gtFine/val
0 val/frankfurt/frankfurt_000000_000294_leftImg8bit.png
1 val/frankfurt/frankfurt_000000_000576_leftImg8bit.png
2 val/frankfurt/frankfurt_000000_001016_leftImg8bit.png
3 val/frankfurt/frankfurt_000000_001236_leftImg8bit.png
4 val/frankfurt/frankfurt_000000_001751_leftImg8bit.png
5 val/frankfurt/frankfurt_000000_002196_leftImg8bit.png
6 val/frankfurt/frankfurt_000000_002963_leftImg8bit.png
7 val/frankfurt/frankfurt_000000_003025_leftImg8bit.png
8 val/frankfurt/frankfurt_000000_003357_leftImg8bit.png
9 val/frankfurt/frankfurt_000000_003920_leftImg8bit.png
10 val/frankfurt/frankfurt_000000_004617_leftImg8bit.png
11 val/frankfurt/frankfurt_000000_005543_leftImg8bit.png
12 val/frankfurt/frankfurt_000000_005898_leftImg8bit.png
13 val/frankfurt/frankfurt_000000_006589_leftImg8bit.png
14 val/frankfurt/frankfurt_000000_007365_leftImg8bit.png
15 val/frankfurt/frankfurt_000000_008206_leftImg8bit.png
16 val/frankf

In [ ]:
bisenet_to_quantize = copy.deepcopy(bisenet_fp32)
qconfig_mapping = get_default_qconfig_mapping("fbgemm")
bisenet_to_quantize.eval()
# prepare
bisenet_prepared = quantize_fx.prepare_fx(bisenet_to_quantize, qconfig_mapping, calibration_data)
with torch.no_grad():
  for idx, image in enumerate(calibration_data):
    print(idx)
    bisenet_prepared(image)
# quantize
bisenet_ptsq_fbgemm = quantize_fx.convert_fx(bisenet_prepared)

In [ ]:
# @title
import numpy as np
import torch
import torch.nn.functional as F
import os
import importlib
import time

from PIL import Image
from argparse import ArgumentParser

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize
from torchvision.transforms import ToTensor, ToPILImage

from transform import Relabel, ToLabel, Colorize
from iouEval import iouEval, getColorEntry

NUM_CHANNELS = 3
NUM_CLASSES = 20

image_transform = ToPILImage()
input_transform_cityscapes = Compose([
    Resize(512, Image.BILINEAR),
    ToTensor(),
])
target_transform_cityscapes = Compose([
    Resize(512, Image.NEAREST),
    ToLabel(),
    Relabel(255, 19),   #ignore label to 19
])

loader = DataLoader(cityscapes('/content/Datasets/Cityscapes/', input_transform=input_transform_cityscapes, target_transform=target_transform_cityscapes, subset='val'), num_workers=2, batch_size=1, shuffle=False)

iouEvalVal = iouEval(NUM_CLASSES)

start = time.time()
bisenet_ptsq_fbgemm.to(torch.device('cpu'))
bisenet_ptsq_fbgemm.eval()
bisenet_ptsq_fbgemm.aux_mode='eval'


i=0
for step, (images, labels, filename, filenameGt) in enumerate(loader):

    inputs = Variable(images)
    #inputs_8bit = torch.quantize_per_tensor(inputs, 0.1, 10, torch.quint8)
    with torch.no_grad():
        outputs = bisenet_ptsq_fbgemm(inputs)
        #fp32_outputs = fp32_model(inputs)
    iouEvalVal.addBatch(outputs[1].max(1)[1].unsqueeze(1).data, labels)

    filenameSave = filename[0].split("leftImg8bit/")[1]

    print (step, filenameSave)
    i +=1
iouVal, iou_classes = iouEvalVal.getIoU()

iou_classes_str = []
for i in range(iou_classes.size(0)):
    iouStr = getColorEntry(iou_classes[i])+'{:0.2f}'.format(iou_classes[i]*100) + '\033[0m'
    iou_classes_str.append(iouStr)

print("---------------------------------------")
print("Took ", time.time()-start, "seconds")
print("=======================================")
#print("TOTAL IOU: ", iou * 100, "%")
print("Per-Class IoU:")
print(iou_classes_str[0], "Road")
print(iou_classes_str[1], "sidewalk")
print(iou_classes_str[2], "building")
print(iou_classes_str[3], "wall")
print(iou_classes_str[4], "fence")
print(iou_classes_str[5], "pole")
print(iou_classes_str[6], "traffic light")
print(iou_classes_str[7], "traffic sign")
print(iou_classes_str[8], "vegetation")
print(iou_classes_str[9], "terrain")
print(iou_classes_str[10], "sky")
print(iou_classes_str[11], "person")
print(iou_classes_str[12], "rider")
print(iou_classes_str[13], "car")
print(iou_classes_str[14], "truck")
print(iou_classes_str[15], "bus")
print(iou_classes_str[16], "train")
print(iou_classes_str[17], "motorcycle")
print(iou_classes_str[18], "bicycle")
print("=======================================")
iouStr = getColorEntry(iouVal)+'{:0.2f}'.format(iouVal*100) + '\033[0m'
print ("MEAN IoU: ", iouStr, "%")

/content/Datasets/Cityscapes/leftImg8bit/val /content/Datasets/Cityscapes/gtFine/val
0 val/frankfurt/frankfurt_000000_000294_leftImg8bit.png
1 val/frankfurt/frankfurt_000000_000576_leftImg8bit.png
2 val/frankfurt/frankfurt_000000_001016_leftImg8bit.png
3 val/frankfurt/frankfurt_000000_001236_leftImg8bit.png
4 val/frankfurt/frankfurt_000000_001751_leftImg8bit.png
5 val/frankfurt/frankfurt_000000_002196_leftImg8bit.png
6 val/frankfurt/frankfurt_000000_002963_leftImg8bit.png
7 val/frankfurt/frankfurt_000000_003025_leftImg8bit.png
8 val/frankfurt/frankfurt_000000_003357_leftImg8bit.png
9 val/frankfurt/frankfurt_000000_003920_leftImg8bit.png
10 val/frankfurt/frankfurt_000000_004617_leftImg8bit.png
11 val/frankfurt/frankfurt_000000_005543_leftImg8bit.png
12 val/frankfurt/frankfurt_000000_005898_leftImg8bit.png
13 val/frankfurt/frankfurt_000000_006589_leftImg8bit.png
14 val/frankfurt/frankfurt_000000_007365_leftImg8bit.png
15 val/frankfurt/frankfurt_000000_008206_leftImg8bit.png
16 val/frankf

In [ ]:
iteration_times_q = []
for _ in range(num_iterations):
    start_time = time.time()
    bisenet_ptsq_fbgemm(image)
    end_time = time.time()
    iteration_times_q.append(end_time - start_time)

# Calculate the average time per iteration
average_time_per_iteration = np.mean(iteration_times_q)

# Calculate the standard deviation
std_deviation = np.std(iteration_times_q)

print(f"QUANTIZED Average time per iteration: {average_time_per_iteration} seconds")
print(f"QUANTIZED Standard deviation: {std_deviation} seconds")

QUANTIZED Average time per iteration: 0.5165547752380371 seconds
QUANTIZED Standard deviation: 0.05761676712865999 seconds


### QAT
We decided to use **Quantization Aware Training**, so we can let the network adapt to quantization (rounding, for example) and achieve better results.

In [ ]:
# configuring the QAT process and backend
qconfig_mapping = get_default_qat_qconfig_mapping('x86')
torch.backends.quantized.engine = 'x86'
qconfig_mapping

QConfigMapping (
 global_qconfig
  QConfig(activation=functools.partial(<class 'torch.ao.quantization.fake_quantize.FusedMovingAvgObsFakeQuantize'>, observer=<class 'torch.ao.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.fake_quantize.FusedMovingAvgObsFakeQuantize'>, observer=<class 'torch.ao.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric){})
 object_type_qconfigs
  reshape: QConfig(activation=<class 'torch.ao.quantization.observer.ReuseInputObserver'>, weight=<class 'torch.ao.quantization.observer.NoopObserver'>)
  <class 'torch.nn.modules.conv.ConvTranspose1d'>: QConfig(activation=functools.partial(<class 'torch.ao.quantization.fake_quantize.FusedMovingAvgObsFakeQuantize'>, observer=<class 'torch.ao.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max

In [ ]:
%cd /content/AnomalySegmentation_AML/eval

/content/AnomalySegmentation_AML/eval


In [ ]:
import torch
from torchsummary import summary
from bisenetv1 import BiSeNetV1

# Create an instance of BiSeNet model
model = BiSeNetV1(n_classes=20)
state_dict = torch.load('/content/AnomalySegmentation_AML/trained_models/bisenet_finetuned_final.pth', map_location=torch.device('cpu'))

model.load_state_dict(state_dict)
model.eval()
# Use GPU
#device = torch.device("cuda") # "cpu" or "cuda"
#model.to(device)

# use torchsummary for an overview of the model (structure, params and size)
#summary(model, input_size=(3,480,480))
#print(model)

BiSeNetV1(
  (cp): ContextPath(
    (resnet): Resnet18(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNo

In [ ]:
model_to_quantize = copy.deepcopy(model)
model_to_quantize.train()
#calibration_data = [torch.randn(1,3,480,480)]
model_prepared = prepare_qat_fx(model_to_quantize, qconfig_mapping, (next(iter(calibration_data))[0]))

/usr/local/lib/python3.10/dist-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [ ]:
model_prepared

GraphModule(
  (activation_post_process_0): FusedMovingAvgObsFakeQuantize(
    fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
    (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
  )
  (cp): Module(
    (resnet): Module(
      (conv1): ConvBnReLU2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
          fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, qscheme=torch.per_channel_symmetric, reduce_range=False
          (activation_post_process): MovingAveragePerChannelMinMaxOb

now we have to train the model to let it understand the quantization effect, so it can compensate for that.

We set a low starting lr of 0.0001 and let him cook for some epochs (10).

In [ ]:
%cd /content/AnomalySegmentation_AML/train

/content/AnomalySegmentation_AML/train


In [ ]:
# @title
import os
import random
import time
import numpy as np
import torch
import math
from PIL import Image, ImageOps
from argparse import ArgumentParser
from torch.optim import SGD, Adam, lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize, Pad
from torchvision.transforms import ToTensor, ToPILImage
from dataset import VOC12,cityscapes
from transform import Relabel, ToLabel, Colorize
from visualize import Dashboard
import importlib
from iouEval import iouEval, getColorEntry
from shutil import copyfile

NUM_CHANNELS = 3
NUM_CLASSES = 20 #pascal=22, cityscapes=20
color_transform = Colorize(NUM_CLASSES)
image_transform = ToPILImage()
#Augmentations - different function implemented to perform random augments on both image and target
class MyCoTransform(object):
    def __init__(self, enc, augment=True, height=512):
        self.enc=enc
        self.augment = augment
        self.height = height
        pass
    def __call__(self, input, target):
        # do something to both images
        input =  Resize(self.height, Image.BILINEAR)(input)
        target = Resize(self.height, Image.NEAREST)(target)
        if(self.augment):
            # Random hflip
            hflip = random.random()
            if (hflip < 0.5):
                input = input.transpose(Image.FLIP_LEFT_RIGHT)
                target = target.transpose(Image.FLIP_LEFT_RIGHT)
            #Random translation 0-2 pixels (fill rest with padding
            transX = random.randint(-2, 2)
            transY = random.randint(-2, 2)
            input = ImageOps.expand(input, border=(transX,transY,0,0), fill=0)
            target = ImageOps.expand(target, border=(transX,transY,0,0), fill=255) #pad label filling with 255
            input = input.crop((0, 0, input.size[0]-transX, input.size[1]-transY))
            target = target.crop((0, 0, target.size[0]-transX, target.size[1]-transY))
        input = ToTensor()(input)
        if (self.enc):
            target = Resize(int(self.height/8), Image.NEAREST)(target)
        target = ToLabel()(target)
        target = Relabel(255, 19)(target)
        return input, target


class CrossEntropyLoss2d(torch.nn.Module):
    def __init__(self, weight=None):
        super().__init__()
        self.loss = torch.nn.NLLLoss2d(weight)
    def forward(self, outputs, targets):
        return self.loss(torch.nn.functional.log_softmax(outputs, dim=1), targets)

def train(args, model, enc=False):
    best_acc = 0
    if args['cuda']:
      model.cuda()
    #TODO: calculate weights by processing dataset histogram (now its being set by hand from the torch values)
    #create a loder to run all images and calculate histogram of labels, then create weight array using class balancing

    weight = torch.ones(NUM_CLASSES)
    if (enc):
        weight[0] = 2.3653597831726
        weight[1] = 4.4237880706787
        weight[2] = 2.9691488742828
        weight[3] = 5.3442072868347
        weight[4] = 5.2983593940735
        weight[5] = 5.2275490760803
        weight[6] = 5.4394111633301
        weight[7] = 5.3659925460815
        weight[8] = 3.4170460700989
        weight[9] = 5.2414722442627
        weight[10] = 4.7376127243042
        weight[11] = 5.2286224365234
        weight[12] = 5.455126285553
        weight[13] = 4.3019247055054
        weight[14] = 5.4264230728149
        weight[15] = 5.4331531524658
        weight[16] = 5.433765411377
        weight[17] = 5.4631009101868
        weight[18] = 5.3947434425354
    else:
        weight[0] = 2.8149201869965
        weight[1] = 6.9850029945374
        weight[2] = 3.7890393733978
        weight[3] = 9.9428062438965
        weight[4] = 9.7702074050903
        weight[5] = 9.5110931396484
        weight[6] = 10.311357498169
        weight[7] = 10.026463508606
        weight[8] = 4.6323022842407
        weight[9] = 9.5608062744141
        weight[10] = 7.8698215484619
        weight[11] = 9.5168733596802
        weight[12] = 10.373730659485
        weight[13] = 6.6616044044495
        weight[14] = 10.260489463806
        weight[15] = 10.287888526917
        weight[16] = 10.289801597595
        weight[17] = 10.405355453491
        weight[18] = 10.138095855713

    weight[19] = 1.0

    assert os.path.exists(args['datadir']), "Error: datadir (dataset directory) could not be loaded"

    co_transform = MyCoTransform(enc, augment=True, height=args['height'])#1024)
    co_transform_val = MyCoTransform(enc, augment=False, height=args['height'])#1024)
    dataset_train = cityscapes(args['datadir'], co_transform, 'train')
    dataset_val = cityscapes(args['datadir'], co_transform_val, 'val')

    loader = DataLoader(dataset_train, num_workers=args['num_workers'], batch_size=args['batch_size'], shuffle=True)
    loader_val = DataLoader(dataset_val, num_workers=args['num_workers'], batch_size=args['batch_size'], shuffle=False)

    if args['cuda']:
        weight = weight.cuda()
    criterion = CrossEntropyLoss2d(weight)
    print(type(criterion))

    savedir = args['savedir']

    if (enc):
        automated_log_path = savedir + "/automated_log_encoder.txt"
        modeltxtpath = savedir + "/model_encoder.txt"
    else:
        automated_log_path = savedir + "/automated_log.txt"
        modeltxtpath = savedir + "/model.txt"

    if (not os.path.exists(automated_log_path)):    #dont add first line if it exists
        with open(automated_log_path, "a") as myfile:
            myfile.write("Epoch\t\tTrain-loss\t\tTest-loss\t\tTrain-IoU\t\tTest-IoU\t\tlearningRate")

    with open(modeltxtpath, "w") as myfile:
        myfile.write(str(model))


    #optimizer = Adam(model.parameters(), 5e-4, (0.9, 0.999),  eps=1e-08, weight_decay=2e-4)     ## scheduler 1
    optimizer = Adam(list(model.parameters()), 1e-4, (0.9, 0.999),  eps=1e-08, weight_decay=1e-4)      ## scheduler 2

    start_epoch = 1
    if args['resume']:
        #Must load weights, optimizer, epoch and best value.
        if enc:
            filenameCheckpoint = savedir + '/checkpoint_enc.pth.tar'
        else:
            filenameCheckpoint = savedir + '/checkpoint.pth.tar'

        assert os.path.exists(filenameCheckpoint), "Error: resume option was used but checkpoint was not found in folder"
        checkpoint = torch.load(filenameCheckpoint)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        best_acc = checkpoint['best_acc']
        print("=> Loaded checkpoint at epoch {})".format(checkpoint['epoch']))

    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5) # set up scheduler     ## scheduler 1
    lambda1 = lambda epoch: pow((1-((epoch-1)/args['num_epochs'])),0.9)  ## scheduler 2
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)                             ## scheduler 2

    if args['visualize'] and args['steps_plot'] > 0:
        board = Dashboard(8097)

    for epoch in range(start_epoch, args['num_epochs']+1):
        print("----- TRAINING - EPOCH", epoch, "-----")
        #print(weight)
        scheduler.step(epoch)    ## scheduler 2

        epoch_loss = []
        time_train = []

        doIouTrain = args['iouTrain']
        doIouVal =  args['iouVal']

        if (doIouTrain):
            iouEvalTrain = iouEval(NUM_CLASSES)

        usedLr = 0
        for param_group in optimizer.param_groups:
            print("LEARNING RATE: ", param_group['lr'])
            usedLr = float(param_group['lr'])

        model.train()
        for step, (images, labels) in enumerate(loader):

            start_time = time.time()

            if args['cuda']:
                images = images.cuda()
                labels = labels.cuda()

            inputs = Variable(images)
            targets = Variable(labels)
            outputs, *_ = model(inputs)
            #print(outputs.shape)


            optimizer.zero_grad()
            loss = criterion(outputs, targets[:, 0])
            loss.backward()
            optimizer.step()

            epoch_loss.append(loss.data.item())
            time_train.append(time.time() - start_time)

            if (doIouTrain):
                iouEvalTrain.addBatch(outputs.max(1)[1].unsqueeze(1).data, targets.data)

            #print(outputs.size())
            if args['visualize'] and args['steps_plot'] > 0 and step % args['steps_plot'] == 0:
                start_time_plot = time.time()
                image = inputs[0].cpu().data

                board.image(image, f'input (epoch: {epoch}, step: {step})')
                if isinstance(outputs, list):   #merge gpu tensors
                    board.image(color_transform(outputs[0][0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'output (epoch: {epoch}, step: {step})')
                else:
                    board.image(color_transform(outputs[0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'output (epoch: {epoch}, step: {step})')
                board.image(color_transform(targets[0].cpu().data),
                    f'target (epoch: {epoch}, step: {step})')
                print ("Time to paint images: ", time.time() - start_time_plot)
            if args['steps_loss'] > 0 and step % args['steps_loss'] == 0:
                average = sum(epoch_loss) / len(epoch_loss)
                print(f'loss: {average:0.4} (epoch: {epoch}, step: {step})',
                        "// Avg time/img: %.4f s" % (sum(time_train) / len(time_train) / args['batch_size']))


        average_epoch_loss_train = sum(epoch_loss) / len(epoch_loss)

        iouTrain = 0
        if (doIouTrain):
            iouTrain, iou_classes = iouEvalTrain.getIoU()
            iouStr = getColorEntry(iouTrain)+'{:0.2f}'.format(iouTrain*100) + '\033[0m'
            print ("EPOCH IoU on TRAIN set: ", iouStr, "%")

        #Validate on 500 val images after each epoch of training
        print("----- VALIDATING - EPOCH", epoch, "-----")
        model.eval()
        epoch_loss_val = []
        time_val = []

        if (doIouVal):
            iouEvalVal = iouEval(NUM_CLASSES)

        for step, (images, labels) in enumerate(loader_val):
            start_time = time.time()
            if args['cuda']:
                images = images.cuda()
                labels = labels.cuda()

            inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
            targets = Variable(labels, volatile=True)
            outputs, *_ = model(inputs)

            loss = criterion(outputs, targets[:, 0])
            epoch_loss_val.append(loss.data.item())
            time_val.append(time.time() - start_time)


            #Add batch to calculate TP, FP and FN for iou estimation
            if (doIouVal):
                iouEvalVal.addBatch(outputs.max(1)[1].unsqueeze(1).data, targets.data)

            if args['visualize'] and args['steps_plot'] > 0 and step % args['steps_plot'] == 0:
                start_time_plot = time.time()
                image = inputs[0].cpu().data
                board.image(image, f'VAL input (epoch: {epoch}, step: {step})')
                if isinstance(outputs, list):   #merge gpu tensors
                    board.image(color_transform(outputs[0][0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'VAL output (epoch: {epoch}, step: {step})')
                else:
                    board.image(color_transform(outputs[0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'VAL output (epoch: {epoch}, step: {step})')
                board.image(color_transform(targets[0].cpu().data),
                    f'VAL target (epoch: {epoch}, step: {step})')
                print ("Time to paint images: ", time.time() - start_time_plot)
            if args['steps_loss'] > 0 and step % args['steps_loss'] == 0:
                average = sum(epoch_loss_val) / len(epoch_loss_val)
                print(f'VAL loss: {average:0.4} (epoch: {epoch}, step: {step})',
                        "// Avg time/img: %.4f s" % (sum(time_val) / len(time_val) / args['batch_size']))


        average_epoch_loss_val = sum(epoch_loss_val) / len(epoch_loss_val)

        iouVal = 0
        if (doIouVal):
            iouVal, iou_classes = iouEvalVal.getIoU()
            iouStr = getColorEntry(iouVal)+'{:0.2f}'.format(iouVal*100) + '\033[0m'
            print ("EPOCH IoU on VAL set: ", iouStr, "%")


        if iouVal == 0:
            current_acc = -average_epoch_loss_val
        else:
            current_acc = iouVal
        is_best = current_acc > best_acc
        best_acc = max(current_acc, best_acc)
        if enc:
            filenameCheckpoint = savedir + '/checkpoint_enc.pth.tar'
            filenameBest = savedir + '/model_best_enc.pth.tar'
        else:
            filenameCheckpoint = savedir + '/checkpoint.pth.tar'
            filenameBest = savedir + '/model_best.pth.tar'
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': str(model),
            'state_dict': model.state_dict(),
            'best_acc': best_acc,
            'optimizer' : optimizer.state_dict(),
        }, is_best, filenameCheckpoint, filenameBest)

        #SAVE MODEL AFTER EPOCH
        if (enc):
            filename = f'{savedir}/model_encoder-{epoch:03}.pth'
            filenamebest = f'{savedir}/model_encoder_best.pth'
        else:
            filename = f'{savedir}/model-{epoch:03}.pth'
            filenamebest = f'{savedir}/model_best.pth'
        if args['epochs_save'] > 0 and step > 0 and step % args['epochs_save'] == 0:
            torch.save(model.state_dict(), filename)
            print(f'save: {filename} (epoch: {epoch})')
        if (is_best):
            torch.save(model.state_dict(), filenamebest)
            print(f'save: {filenamebest} (epoch: {epoch})')
            if (not enc):
                with open(savedir + "/best.txt", "w") as myfile:
                    myfile.write("Best epoch is %d, with Val-IoU= %.4f" % (epoch, iouVal))
            else:
                with open(savedir + "/best_encoder.txt", "w") as myfile:
                    myfile.write("Best epoch is %d, with Val-IoU= %.4f" % (epoch, iouVal))

        #SAVE TO FILE A ROW WITH THE EPOCH RESULT (train loss, val loss, train IoU, val IoU)
        #Epoch		Train-loss		Test-loss	Train-IoU	Test-IoU		learningRate
        with open(automated_log_path, "a") as myfile:
            myfile.write("\n%d\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.8f" % (epoch, average_epoch_loss_train, average_epoch_loss_val, iouTrain, iouVal, usedLr ))

    return(model)   #return model (convenience for encoder-decoder training)

def save_checkpoint(state, is_best, filenameCheckpoint, filenameBest):
    torch.save(state, filenameCheckpoint)
    if is_best:
        print ("Saving model as best")
        torch.save(state, filenameBest)

In [ ]:
import logging
import logging.handlers
import sys


savedir = f'../save_quantized/BiSeNet_pruned'
if not os.path.exists(savedir):
        os.makedirs(savedir)

args = {
    'height': 512,
    'num_workers': 2,
    'batch_size' : 6,
    'cuda' : True,
    'savedir' : savedir,
    'num_epochs': 5,
    'steps_plot' : 50,
    'visualize' : False,
    'iouTrain': False,
    'iouVal': True,
    'steps_loss' : 50,
    'epochs_save': 0,
    'datadir': f'/content/Datasets/Cityscapes/',
    'resume': False
}
model_prepared.cuda()
model_trained_prepared = train(args, model_prepared)

/content/Datasets/Cityscapes/leftImg8bit/train
/content/Datasets/Cityscapes/leftImg8bit/val
<class '__main__.CrossEntropyLoss2d'>


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:222: UserWarning: NLLLoss2d has been deprecated. Please use NLLLoss instead as a drop-in replacement and see https://pytorch.org/docs/master/nn.html#torch.nn.NLLLoss for more details.
  warnings.warn("NLLLoss2d has been deprecated. "


----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.0001


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://githu

loss: 0.2692 (epoch: 1, step: 0) // Avg time/img: 0.0927 s
loss: 0.2777 (epoch: 1, step: 50) // Avg time/img: 0.0838 s
loss: 0.2693 (epoch: 1, step: 100) // Avg time/img: 0.0847 s
loss: 0.2627 (epoch: 1, step: 150) // Avg time/img: 0.0851 s
loss: 0.254 (epoch: 1, step: 200) // Avg time/img: 0.0852 s
loss: 0.2436 (epoch: 1, step: 250) // Avg time/img: 0.0852 s
loss: 0.2366 (epoch: 1, step: 300) // Avg time/img: 0.0852 s
loss: 0.2306 (epoch: 1, step: 350) // Avg time/img: 0.0852 s
loss: 0.2248 (epoch: 1, step: 400) // Avg time/img: 0.0854 s
loss: 0.2208 (epoch: 1, step: 450) // Avg time/img: 0.0853 s
----- VALIDATING - EPOCH 1 -----


<ipython-input-197-48a603d358d1>:262: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
<ipython-input-197-48a603d358d1>:263: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  targets = Variable(labels, volatile=True)


VAL loss: 0.1915 (epoch: 1, step: 0) // Avg time/img: 0.0706 s
VAL loss: 0.393 (epoch: 1, step: 50) // Avg time/img: 0.0373 s
EPOCH IoU on VAL set:  65.03 %
Saving model as best
save: ../save_quantized/BiSeNet_pruned/model_best.pth (epoch: 1)
----- TRAINING - EPOCH 2 -----
LEARNING RATE:  8.180521460508584e-05
loss: 0.2076 (epoch: 2, step: 0) // Avg time/img: 0.0988 s
loss: 0.1775 (epoch: 2, step: 50) // Avg time/img: 0.0850 s
loss: 0.1716 (epoch: 2, step: 100) // Avg time/img: 0.0849 s
loss: 0.1701 (epoch: 2, step: 150) // Avg time/img: 0.0855 s
loss: 0.1675 (epoch: 2, step: 200) // Avg time/img: 0.0856 s
loss: 0.1677 (epoch: 2, step: 250) // Avg time/img: 0.0856 s
loss: 0.1678 (epoch: 2, step: 300) // Avg time/img: 0.0855 s
loss: 0.1676 (epoch: 2, step: 350) // Avg time/img: 0.0853 s
loss: 0.168 (epoch: 2, step: 400) // Avg time/img: 0.0853 s
loss: 0.169 (epoch: 2, step: 450) // Avg time/img: 0.0853 s
----- VALIDATING - EPOCH 2 -----
VAL loss: 0.1886 (epoch: 2, step: 0) // Avg time/i

In [ ]:
# @title
import logging
import logging.handlers
import sys


savedir = f'../save_quantized/BiSeNet_pruned'
if not os.path.exists(savedir):
        os.makedirs(savedir)

args = {
    'height': 512,
    'num_workers': 2,
    'batch_size' : 4,
    'cuda' : True,
    'savedir' : savedir,
    'num_epochs': 1,
    'steps_plot' : 50,
    'visualize' : False,
    'iouTrain': False,
    'iouVal': True,
    'steps_loss' : 50,
    'epochs_save': 0,
    'datadir': f'/content/Datasets/Cityscapes/',
    'resume': False
}
model_prepared.cuda()
model_trained_prepared = train(args, model_prepared)

/content/Datasets/Cityscapes/leftImg8bit/train
/content/Datasets/Cityscapes/leftImg8bit/val
<class '__main__.CrossEntropyLoss2d'>


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:222: UserWarning: NLLLoss2d has been deprecated. Please use NLLLoss instead as a drop-in replacement and see https://pytorch.org/docs/master/nn.html#torch.nn.NLLLoss for more details.
  warnings.warn("NLLLoss2d has been deprecated. "


----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.0001


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://githu

loss: 0.1598 (epoch: 1, step: 0) // Avg time/img: 0.6044 s
loss: 0.1776 (epoch: 1, step: 50) // Avg time/img: 0.0969 s
loss: 0.1803 (epoch: 1, step: 100) // Avg time/img: 0.0922 s
loss: 0.186 (epoch: 1, step: 150) // Avg time/img: 0.0919 s
loss: 0.1842 (epoch: 1, step: 200) // Avg time/img: 0.0915 s
loss: 0.1859 (epoch: 1, step: 250) // Avg time/img: 0.0915 s
loss: 0.1842 (epoch: 1, step: 300) // Avg time/img: 0.0911 s
loss: 0.1836 (epoch: 1, step: 350) // Avg time/img: 0.0910 s
loss: 0.1839 (epoch: 1, step: 400) // Avg time/img: 0.0911 s
loss: 0.1832 (epoch: 1, step: 450) // Avg time/img: 0.0910 s
loss: 0.1831 (epoch: 1, step: 500) // Avg time/img: 0.0909 s
loss: 0.183 (epoch: 1, step: 550) // Avg time/img: 0.0908 s
loss: 0.1827 (epoch: 1, step: 600) // Avg time/img: 0.0908 s
loss: 0.1831 (epoch: 1, step: 650) // Avg time/img: 0.0907 s
loss: 0.1832 (epoch: 1, step: 700) // Avg time/img: 0.0907 s
----- VALIDATING - EPOCH 1 -----


<ipython-input-17-cc7a47064c81>:263: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
<ipython-input-17-cc7a47064c81>:264: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  targets = Variable(labels, volatile=True)


VAL loss: 0.3127 (epoch: 1, step: 0) // Avg time/img: 0.0590 s
VAL loss: 0.4002 (epoch: 1, step: 50) // Avg time/img: 0.0429 s
VAL loss: 0.3952 (epoch: 1, step: 100) // Avg time/img: 0.0426 s
EPOCH IoU on VAL set:  62.77 %
Saving model as best
save: ../save_quantized/BiSeNet_pruned/model_best.pth (epoch: 1)


In [ ]:
model_trained_prepared.cpu()
model_quantized = convert_fx(model_trained_prepared)
torch.save(model_quantized.state_dict(), "/content/AnomalySegmentation_AML/trained_models/bisenet_qat.pth")
print('Bisenet (QUANTIZED INT8) Size (MB):', os.path.getsize("/content/AnomalySegmentation_AML/trained_models/bisenet_qat.pth")/1e6)

Bisenet (QUANTIZED INT8) Size (MB): 13.63401


In [ ]:
torch.save(model_quantized, '/content/AnomalySegmentation_AML/trained_models/MODEL_bisenet_qat.pth')

In [ ]:
model_quantized.cpu()
model_quantized.eval()

GraphModule(
  (cp): Module(
    (resnet): Module(
      (conv1): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.07064152508974075, zero_point=0, padding=(3, 3))
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Module(
        (0): Module(
          (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.05754431337118149, zero_point=0, padding=(1, 1))
          (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.12356444448232651, zero_point=69, padding=(1, 1))
        )
        (1): Module(
          (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.045720018446445465, zero_point=0, padding=(1, 1))
          (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.12891419231891632, zero_point=76, padding=(1, 1))
        )
      )
      (layer2): Module(
        (0): Module(
          (conv1): QuantizedConvR

In [ ]:
import time
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import FakeData
import numpy as np

def inference_time(model, dataloader, device):
    model.eval()
    total_time = 0.0
    num_samples = 0

    with torch.no_grad():
        for inputs in dataloader:
            image = inputs[0].cpu()

            start_time = time.time()
            _ = model(image)  # Perform inference, adjust as needed based on your model's forward pass
            end_time = time.time()

            total_time += end_time - start_time
            num_samples += 1

    avg_time = total_time / num_samples
    return avg_time

# Set a random seed for reproducibility
torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a data loader with random images
transform = transforms.Compose([transforms.ToTensor()])
fake_dataset = FakeData(transform=transform, image_size= (3,480,480), size=60)
test_loader = DataLoader(fake_dataset, batch_size=1, shuffle=True, num_workers=2)

In [ ]:
device = torch.device("cpu")
inference_times_q = []
for run in range(3):
    time_model_q = inference_time(model_quantized, test_loader, device)
    inference_times_q.append(time_model_q)
    print(f"Run {run + 1}: Inference time for quantized model: {time_model_q:.6f} seconds per sample")

# Calculate average and standard deviation
avg_time_model_q = np.mean(inference_times_q)
std_dev_model_q = np.std(inference_times_q)

print(f"\nAverage inference time for quantized model: {avg_time_model_q:.6f} seconds per sample")
print(f"Standard deviation for quantized model: {std_dev_model_q:.6f} seconds per sample\n")


Run 1: Inference time for quantized model: 0.396352 seconds per sample
Run 2: Inference time for quantized model: 0.392651 seconds per sample
Run 3: Inference time for quantized model: 0.398353 seconds per sample

Average inference time for quantized model: 0.395785 seconds per sample
Standard deviation for quantized model: 0.002362 seconds per sample



In [ ]:
bisenet_full = BiSeNetV1(n_classes=20)
state_dict = torch.load('/content/AnomalySegmentation_AML/trained_models/bisenet_finetuned_final.pth', map_location=torch.device('cpu')) # , map_location=lambda storage, loc: storage)

bisenet_full.load_state_dict(state_dict)
bisenet_full.aux_mode = 'eval'
bisenet_full.eval()

In [ ]:
device = torch.device("cpu")
inference_times_q = []
for run in range(3):
    time_model_q = inference_time(bisenet_full, test_loader, device)
    inference_times_q.append(time_model_q)
    print(f"Run {run + 1}: Inference time for baseline model: {time_model_q:.6f} seconds per sample")

# Calculate average and standard deviation
avg_time_model_q = np.mean(inference_times_q)
std_dev_model_q = np.std(inference_times_q)

print(f"\nAverage inference time for baseline model: {avg_time_model_q:.6f} seconds per sample")
print(f"Standard deviation for baseline model: {std_dev_model_q:.6f} seconds per sample\n")


Run 1: Inference time for baseline model: 3.614130 seconds per sample
Run 2: Inference time for baseline model: 3.532710 seconds per sample
Run 3: Inference time for baseline model: 3.530459 seconds per sample

Average inference time for baseline model: 3.559100 seconds per sample
Standard deviation for baseline model: 0.038923 seconds per sample



In [ ]:
pruned_model = BiSeNetV1(n_classes=20)
state_dict = torch.load('/content/AnomalySegmentation_AML/trained_models/bisenet_finetuned_after_pruning.pth', map_location=torch.device('cpu')) # , map_location=lambda storage, loc: storage)

pruned_model.load_state_dict(state_dict)
pruned_model.aux_mode = 'eval'
pruned_model.eval()

In [ ]:
device = torch.device("cpu")
inference_times_p = []

for run in range(3):
    time_model_p = inference_time(pruned_model, test_loader, device)
    inference_times_p.append(time_model_p)
    print(f"Run {run + 1}: Inference time for baseline model: {time_model_p:.6f} seconds per sample")

# Calculate average and standard deviation
avg_time_model_p = np.mean(inference_times_p)
std_dev_model_p = np.std(inference_times_p)

print(f"\nAverage inference time for baseline model: {avg_time_model_p:.6f} seconds per sample")
print(f"Standard deviation for baseline model: {std_dev_model_p:.6f} seconds per sample\n")


Run 1: Inference time for baseline model: 1.604319 seconds per sample
Run 2: Inference time for baseline model: 1.589490 seconds per sample
Run 3: Inference time for baseline model: 1.587083 seconds per sample

Average inference time for baseline model: 1.593631 seconds per sample
Standard deviation for baseline model: 0.007621 seconds per sample



we run the code almost 10 TIMES FASTER!


In [ ]:
 0.116448/0.064330

1.8101663298616508

In [ ]:
import io
b = io.BytesIO()

In [ ]:
torch.save(model_quantized.state_dict(), b)

In [ ]:
b.seek(0)

0

In [ ]:
file_path = "/content/AnomalySegmentation_AML/trained_models/bisenet_qat_bytesio"
with open(file_path, "wb") as file:
    file.write(b.getvalue())

In [ ]:
%cd /content/AnomalySegmentation_AML/eval

/content/AnomalySegmentation_AML/eval


In [ ]:
# @title
# Code with dataset loader for VOC12 and Cityscapes (adapted from bodokaiser/piwise code)
# Sept 2017
# Eduardo Romera
#######################

import numpy as np
import os

from PIL import Image

from torch.utils.data import Dataset

EXTENSIONS = ['.jpg', '.png']

def load_image(file):
    return Image.open(file)

def is_image(filename):
    return any(filename.endswith(ext) for ext in EXTENSIONS)

def is_label(filename):
    return filename.endswith("_labelTrainIds.png")

def image_path(root, basename, extension):
    return os.path.join(root, f'{basename}{extension}')

def image_path_city(root, name):
    return os.path.join(root, f'{name}')

def image_basename(filename):
    return os.path.basename(os.path.splitext(filename)[0])

class VOC12(Dataset):

    def __init__(self, root, input_transform=None, target_transform=None):
        self.images_root = os.path.join(root, 'images')
        self.labels_root = os.path.join(root, 'labels')

        self.filenames = [image_basename(f)
            for f in os.listdir(self.labels_root) if is_image(f)]
        self.filenames.sort()

        self.input_transform = input_transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        filename = self.filenames[index]

        with open(image_path(self.images_root, filename, '.jpg'), 'rb') as f:
            image = load_image(f).convert('RGB')
        with open(image_path(self.labels_root, filename, '.png'), 'rb') as f:
            label = load_image(f).convert('P')

        if self.input_transform is not None:
            image = self.input_transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)

        return image, label

    def __len__(self):
        return len(self.filenames)


class cityscapes(Dataset):

    def __init__(self, root, input_transform=None, target_transform=None, subset='val'):

        self.images_root = os.path.join(root, 'leftImg8bit/' + subset)
        self.labels_root = os.path.join(root, 'gtFine/' + subset)
        print(self.images_root, self.labels_root)
        self.filenames = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(self.images_root)) for f in fn if is_image(f)]
        self.filenames.sort()

        self.filenamesGt = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(self.labels_root)) for f in fn if is_label(f)]
        self.filenamesGt.sort()

        self.input_transform = input_transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        filename = self.filenames[index]
        filenameGt = self.filenamesGt[index]

        #print(filename)

        with open(image_path_city(self.images_root, filename), 'rb') as f:
            image = load_image(f).convert('RGB')
        with open(image_path_city(self.labels_root, filenameGt), 'rb') as f:
            label = load_image(f).convert('P')

        if self.input_transform is not None:
            image = self.input_transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)

        return image, label, filename, filenameGt

    def __len__(self):
        return len(self.filenames)



In [ ]:
# @title
import numpy as np
import torch
import torch.nn.functional as F
import os
import importlib
import time

from PIL import Image
from argparse import ArgumentParser

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize
from torchvision.transforms import ToTensor, ToPILImage

from transform import Relabel, ToLabel, Colorize
from iouEval import iouEval, getColorEntry

NUM_CHANNELS = 3
NUM_CLASSES = 20

image_transform = ToPILImage()
input_transform_cityscapes = Compose([
    Resize(512, Image.BILINEAR),
    ToTensor(),
])
target_transform_cityscapes = Compose([
    Resize(512, Image.NEAREST),
    ToLabel(),
    Relabel(255, 19),   #ignore label to 19
])

loader = DataLoader(cityscapes('/content/Datasets/Cityscapes/', input_transform=input_transform_cityscapes, target_transform=target_transform_cityscapes, subset='val'), num_workers=2, batch_size=1, shuffle=False)

iouEvalVal = iouEval(NUM_CLASSES)

start = time.time()
model_quantized.to(torch.device('cpu'))
model_quantized.eval()
model_quantized.aux_mode='eval'

#fp32_model.to(torch.device('cpu'))
#fp32_model.eval()#
#fp32_model.aux_mode='eval'
i=0
for step, (images, labels, filename, filenameGt) in enumerate(loader):

    inputs = Variable(images)
    #inputs_8bit = torch.quantize_per_tensor(inputs, 0.1, 10, torch.quint8)
    with torch.no_grad():
        outputs = model_quantized(inputs)
        #fp32_outputs = fp32_model(inputs)
    iouEvalVal.addBatch(outputs[1].max(1)[1].unsqueeze(1).data, labels)

    filenameSave = filename[0].split("leftImg8bit/")[1]

    print (step, filenameSave)
    i +=1
iouVal, iou_classes = iouEvalVal.getIoU()

iou_classes_str = []
for i in range(iou_classes.size(0)):
    iouStr = getColorEntry(iou_classes[i])+'{:0.2f}'.format(iou_classes[i]*100) + '\033[0m'
    iou_classes_str.append(iouStr)

print("---------------------------------------")
print("Took ", time.time()-start, "seconds")
print("=======================================")
#print("TOTAL IOU: ", iou * 100, "%")
print("Per-Class IoU:")
print(iou_classes_str[0], "Road")
print(iou_classes_str[1], "sidewalk")
print(iou_classes_str[2], "building")
print(iou_classes_str[3], "wall")
print(iou_classes_str[4], "fence")
print(iou_classes_str[5], "pole")
print(iou_classes_str[6], "traffic light")
print(iou_classes_str[7], "traffic sign")
print(iou_classes_str[8], "vegetation")
print(iou_classes_str[9], "terrain")
print(iou_classes_str[10], "sky")
print(iou_classes_str[11], "person")
print(iou_classes_str[12], "rider")
print(iou_classes_str[13], "car")
print(iou_classes_str[14], "truck")
print(iou_classes_str[15], "bus")
print(iou_classes_str[16], "train")
print(iou_classes_str[17], "motorcycle")
print(iou_classes_str[18], "bicycle")
print("=======================================")
iouStr = getColorEntry(iouVal)+'{:0.2f}'.format(iouVal*100) + '\033[0m'
print ("MEAN IoU: ", iouStr, "%")

/content/Datasets/Cityscapes/leftImg8bit/val /content/Datasets/Cityscapes/gtFine/val
0 val/frankfurt/frankfurt_000000_000294_leftImg8bit.png
1 val/frankfurt/frankfurt_000000_000576_leftImg8bit.png
2 val/frankfurt/frankfurt_000000_001016_leftImg8bit.png
3 val/frankfurt/frankfurt_000000_001236_leftImg8bit.png
4 val/frankfurt/frankfurt_000000_001751_leftImg8bit.png
5 val/frankfurt/frankfurt_000000_002196_leftImg8bit.png
6 val/frankfurt/frankfurt_000000_002963_leftImg8bit.png
7 val/frankfurt/frankfurt_000000_003025_leftImg8bit.png
8 val/frankfurt/frankfurt_000000_003357_leftImg8bit.png
9 val/frankfurt/frankfurt_000000_003920_leftImg8bit.png
10 val/frankfurt/frankfurt_000000_004617_leftImg8bit.png
11 val/frankfurt/frankfurt_000000_005543_leftImg8bit.png
12 val/frankfurt/frankfurt_000000_005898_leftImg8bit.png
13 val/frankfurt/frankfurt_000000_006589_leftImg8bit.png
14 val/frankfurt/frankfurt_000000_007365_leftImg8bit.png
15 val/frankfurt/frankfurt_000000_008206_leftImg8bit.png
16 val/frankf

### EvalAnomaly inference of the QAT model

In [ ]:
%cd /content

/content


In [ ]:
# if the last command doesn't work try:
!pip install gdown

file_id = '1r2eFANvSlcUjxcerjC8l6dRa0slowMpx'
output_file = 'Validation_Dataset.zip'

# Scarica il file utilizzando gdown
!gdown --id $file_id -O $output_file

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1r2eFANvSlcUjxcerjC8l6dRa0slowMpx
From (redirected): https://drive.google.com/uc?id=1r2eFANvSlcUjxcerjC8l6dRa0slowMpx&confirm=t&uuid=1ececdfc-87ad-4b85-9194-7da21d0bd4b7
To: /content/Validation_Dataset.zip
100% 329M/329M [00:04<00:00, 74.0MB/s]


In [ ]:
!unzip /content/Validation_Dataset.zip -d /content

In [ ]:
%cd /content/AnomalySegmentation_AML/eval

/content/AnomalySegmentation_AML/eval


In [ ]:
# @title
import os
import cv2
import glob
import torch
import random
from PIL import Image
import numpy as np
import os.path as osp
from argparse import ArgumentParser
from ood_metrics import fpr_at_95_tpr, calc_metrics, plot_roc, plot_pr,plot_barcode
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score
import torch.nn.functional as F
from torchvision.transforms import Resize


seed = 42

# general reproducibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

NUM_CHANNELS = 3
NUM_CLASSES = 20
# gpu training specific
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

def evalAnomaly(args):
    anomaly_score_list = []
    ood_gts_list = []
    model_quantized.eval()

    for path in glob.glob(os.path.expanduser(str(args['input']))):
          images = torch.from_numpy(np.array(Image.open(path).convert('RGB'))).unsqueeze(0).float()
          images = images.permute(0,3,1,2)
          images = Resize((args['height'], args['height']*2), Image.BILINEAR)(images)
          with torch.no_grad():
              result = model_quantized(images)[0]  # This [0] is useless with aux_mode = 'eval'

          if args['method'] == 'VOID':
            anomaly_result = F.softmax(result.squeeze(0), dim=0).data.cpu().numpy()[-1]

          if args['method'] == 'MSP':
              anomaly_result = 1.0 - np.max(F.softmax(result.squeeze(0), dim=0).data.cpu().numpy(), axis=0)
          elif args['method'] == 'MSPT':
              #MSP with temp scaling
              temp = args['temp'] if args['temp']>0 else 1
              anomaly_result = 1.0 - np.max(F.softmax(result.squeeze(0)/temp, dim=0).data.cpu().numpy(), axis=0)
          elif args['method'] == 'MaxLogit':
              anomaly_result =  1 - np.max(F.normalize(result.squeeze(0), dim=0).data.cpu().numpy(), axis=0)
          elif args['method'] == 'MaxEntropy':
              anomaly_result = torch.div(torch.sum(-F.softmax(result.squeeze(0), dim=0) * F.log_softmax(result.squeeze(0), dim=0), dim=0),
                                        torch.log(torch.tensor(F.softmax(result.squeeze(0), dim=0).size(0)))).data.cpu().numpy()

          pathGT = path.replace("images", "labels_masks")
          if "RoadObsticle21" in pathGT:
            pathGT = pathGT.replace("webp", "png")
          if "fs_static" in pathGT:
            pathGT = pathGT.replace("jpg", "png")
          if "RoadAnomaly" in pathGT:
            pathGT = pathGT.replace("jpg", "png")

          mask = Image.open(pathGT)
          mask = Resize((args['height'], args['height']*2), Image.NEAREST)(mask)
          ood_gts = np.array(mask)

          if "RoadAnomaly" in pathGT:
              ood_gts = np.where((ood_gts==2), 1, ood_gts)

          if "LostAndFound" in pathGT:
              ood_gts = np.where((ood_gts==0), 255, ood_gts)
              ood_gts = np.where((ood_gts==1), 0, ood_gts)
              ood_gts = np.where((ood_gts>1)&(ood_gts<201), 1, ood_gts)

          if "Streethazard" in pathGT:
              ood_gts = np.where((ood_gts==14), 255, ood_gts)
              ood_gts = np.where((ood_gts<20), 0, ood_gts)
              ood_gts = np.where((ood_gts==255), 1, ood_gts)

          if 1 not in np.unique(ood_gts):
              continue
          else:
              ood_gts_list.append(ood_gts)
              anomaly_score_list.append(anomaly_result)
          del result, anomaly_result, ood_gts, mask
          #torch.cuda.empty_cache()

    ood_gts = np.array(ood_gts_list)
    anomaly_scores = np.array(anomaly_score_list)
    print(len(ood_gts), len(anomaly_scores))

    ood_mask = (ood_gts == 1)
    ind_mask = (ood_gts == 0)
    print(len(ood_mask), len(ind_mask))

    ood_out = anomaly_scores[ood_mask]
    ind_out = anomaly_scores[ind_mask]
    #print(len(ood_out), len(ind_out))

    ood_label = np.ones(len(ood_out))
    ind_label = np.zeros(len(ind_out))

    #print(len(ood_label), len(ind_label))
    val_out = np.concatenate((ind_out, ood_out))
    val_label = np.concatenate((ind_label, ood_label))

    #print(len(val_out), len(val_label))
    prc_auc = average_precision_score(val_label, val_out)
    fpr = fpr_at_95_tpr(val_out, val_label)

    print(f'AUPRC score: {prc_auc*100.0}')
    print(f'FPR@TPR95: {fpr*100.0}')

In [ ]:
args = {
        'input': "/content/Validation_Dataset/RoadAnomaly21/images/*.png",
        'subset': "val",  # can be val or train (must have labels)
        'datadir': "/home/shyam/ViT-Adapter/segmentation/data/cityscapes/",
        'num_workers': 4,
        'batch_size': 1,
        'cpu': True,
        'method': "MSP",
        'temp': 0.0,
        'height': 512
    }

evalAnomaly(args)

10 10
10 10
AUPRC score: 14.206466024269435
FPR@TPR95: 93.64908349366073


In [ ]:
args['input'] = "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


60 60
60 60
AUPRC score: 11.801031406631509
FPR@TPR95: 90.10050306367955


In [ ]:
args['input'] = "/content/Validation_Dataset/RoadObsticle21/images/*.webp"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


30 30
30 30
AUPRC score: 2.4015034443929153
FPR@TPR95: 72.89570609043574


In [ ]:
args['input'] = "/content/Validation_Dataset/fs_static/images/*.jpg"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


20 20
20 20
AUPRC score: 10.834105453604456
FPR@TPR95: 54.294167300403785


In [ ]:
args['input'] = "/content/Validation_Dataset/FS_LostFound_full/images/*.png"
args['method'] = "MSP"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


99 99
99 99
AUPRC score: 0.45055232538239903
FPR@TPR95: 61.429299706786956


In [ ]:
args['input'] = "/content/Validation_Dataset/RoadAnomaly21/images/*.png"
args['method'] = "MaxLogit"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


10 10
10 10
AUPRC score: 13.587031983051265
FPR@TPR95: 94.3884337815542


In [ ]:
args['input'] = "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


60 60
60 60
AUPRC score: 10.403300749994859
FPR@TPR95: 88.07295998045626


In [ ]:
args['input'] = "/content/Validation_Dataset/RoadObsticle21/images/*.webp"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


30 30
30 30
AUPRC score: 3.1471405200527593
FPR@TPR95: 92.11918008651413


In [ ]:
args['input'] = "/content/Validation_Dataset/fs_static/images/*.jpg"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


20 20
20 20
AUPRC score: 10.375422105103747
FPR@TPR95: 83.08869541345523


In [ ]:
args['input'] = "/content/Validation_Dataset/FS_LostFound_full/images/*.png"
args['method'] = "MaxLogit"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


99 99
99 99
AUPRC score: 0.30029484260549844
FPR@TPR95: 88.34308998445046


In [ ]:
args['input'] = "/content/Validation_Dataset/RoadAnomaly21/images/*.png"
args['method'] = "MaxEntropy"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


10 10
10 10
AUPRC score: 14.331083682610757
FPR@TPR95: 93.6517107909574


In [ ]:
args['input'] = "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


60 60
60 60
AUPRC score: 11.850531290468467
FPR@TPR95: 90.04078713227122


In [ ]:
args['input'] = "/content/Validation_Dataset/RoadObsticle21/images/*.webp"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


30 30
30 30
AUPRC score: 2.4168350604353157
FPR@TPR95: 72.20912646232726


In [ ]:
args['input'] = "/content/Validation_Dataset/fs_static/images/*.jpg"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


20 20
20 20
AUPRC score: 10.845903129738502
FPR@TPR95: 52.609405323471414


In [ ]:
args['input'] = "/content/Validation_Dataset/FS_LostFound_full/images/*.png"
args['method'] = "MaxEntropy"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


99 99
99 99
AUPRC score: 0.4565653342675351
FPR@TPR95: 62.54863050349061


In [ ]:
args['input'] = "/content/Validation_Dataset/RoadAnomaly21/images/*.png"
args['method'] = "VOID"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


10 10
10 10
AUPRC score: 14.233783656063121
FPR@TPR95: 92.3070945791571


In [ ]:
args['input'] = "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


60 60
60 60
AUPRC score: 10.833581434944492
FPR@TPR95: 89.40232612328583


In [ ]:
args['input'] = "/content/Validation_Dataset/RoadObsticle21/images/*.webp"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


30 30
30 30
AUPRC score: 2.2766378026368232
FPR@TPR95: 82.02603952755071


In [ ]:
args['input'] = "/content/Validation_Dataset/fs_static/images/*.jpg"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


20 20
20 20
AUPRC score: 7.902451263465636
FPR@TPR95: 64.9346069708118


In [ ]:
args['input'] = "/content/Validation_Dataset/FS_LostFound_full/images/*.png"
evalAnomaly(args)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


99 99
99 99
AUPRC score: 0.3523934278616663
FPR@TPR95: 65.54967330487813


# FLOPS COUNTER


In [ ]:
import sys, os
import torch

In [ ]:
from erfnet import ERFNet
from enet import ENet
from bisenetv1 import BiSeNetV1

erfnet = ERFNet(20).eval()
enet = ENet(20).eval()
bisenet = BiSeNetV1(20).eval()

In [ ]:
### check erfnet pruned
erfnet_pruned = ERFNET_PRUNEDV2(20).eval()
erfnet_pruned = add_flops_counting_methods(erfnet_pruned)


In [ ]:
erfnet_pruned.start_flops_count()
_ = erfnet_pruned(torch.randn(1,3,480, 480))
erfnet_pruned.stop_flops_count()

erfnet_pruned.compute_average_flops_cost() / 1e9 / 2 # Result in GFLOPs

16.83654528

In [ ]:
!pip install thop

In [ ]:
from thop import profile
from thop import clever_format
input = torch.randn(1, 3, 480, 480)
macs, params = profile(erfnet, inputs=(input, ))
macs, params = clever_format([macs, params], "%.3f")
macs, params

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.ConvTranspose2d'>.


('13.223G', '2.064M')

In [ ]:
macs, params = profile(erfnet_pruned, inputs=(input, ))
macs, params = clever_format([macs, params], "%.3f")
macs, params

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.ConvTranspose2d'>.


('17.741G', '1.273M')

In [ ]:
# ERFNet full model definition for Pytorch
# Sept 2017
# Eduardo Romera
#######################

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

class DownsamplerBlock (nn.Module):
    def __init__(self, ninput, noutput):
        super().__init__()

        self.conv = nn.Conv2d(ninput, noutput-ninput, (3, 3), stride=2, padding=1, bias=True)
        self.pool = nn.MaxPool2d(2, stride=2)
        self.bn = nn.BatchNorm2d(noutput, eps=1e-3)

    def forward(self, input):
        output = torch.cat([self.conv(input), self.pool(input)], 1)
        output = self.bn(output)
        return F.relu(output)


class non_bottleneck_1d (nn.Module):
    def __init__(self, chann, dropprob, dilated):
        super().__init__()

        self.conv3x1_1 = nn.Conv2d(chann, chann, (3, 1), stride=1, padding=(1,0), bias=True)

        self.conv1x3_1 = nn.Conv2d(chann, chann, (1,3), stride=1, padding=(0,1), bias=True)

        self.bn1 = nn.BatchNorm2d(chann, eps=1e-03)

        self.conv3x1_2 = nn.Conv2d(chann, chann, (3, 1), stride=1, padding=(1*dilated,0), bias=True, dilation = (dilated,1))

        self.conv1x3_2 = nn.Conv2d(chann, chann, (1,3), stride=1, padding=(0,1*dilated), bias=True, dilation = (1, dilated))

        self.bn2 = nn.BatchNorm2d(chann, eps=1e-03)

        self.dropout = nn.Dropout2d(dropprob)


    def forward(self, input):

        output = self.conv3x1_1(input)
        output = F.relu(output)
        output = self.conv1x3_1(output)
        output = self.bn1(output)
        output = F.relu(output)

        output = self.conv3x1_2(output)
        output = F.relu(output)
        output = self.conv1x3_2(output)
        output = self.bn2(output)

        if (self.dropout.p != 0):
            output = self.dropout(output)

        return F.relu(output+input)    #+input = identity (residual connection)


class Encoder(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.initial_block = DownsamplerBlock(3,16)

        self.layers = nn.ModuleList()

        self.layers.append(DownsamplerBlock(16,64))

        for x in range(0, 5):    #5 times
           self.layers.append(non_bottleneck_1d(64, 0.03, 1))

        self.layers.append(DownsamplerBlock(64,128))

        for x in range(0, 2):    #2 times
            self.layers.append(non_bottleneck_1d(128, 0.3, 2))
            self.layers.append(non_bottleneck_1d(128, 0.3, 4))
            self.layers.append(non_bottleneck_1d(128, 0.3, 8))
            self.layers.append(non_bottleneck_1d(128, 0.3, 16))

        #Only in encoder mode:
        self.output_conv = nn.Conv2d(128, num_classes, 1, stride=1, padding=0, bias=True)

    def forward(self, input, predict=False):
        output = self.initial_block(input)

        for layer in self.layers:
            output = layer(output)

        if predict:
            output = self.output_conv(output)

        return output


class UpsamplerBlock (nn.Module):
    def __init__(self, ninput, noutput):
        super().__init__()
        self.conv = nn.ConvTranspose2d(ninput, noutput, 3, stride=2, padding=1, output_padding=1, bias=True)
        self.bn = nn.BatchNorm2d(noutput, eps=1e-3)

    def forward(self, input):
        output = self.conv(input)
        output = self.bn(output)
        return F.relu(output)

class Decoder (nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.layers = nn.ModuleList()

        self.layers.append(UpsamplerBlock(128,32))
        self.layers.append(non_bottleneck_1d(32, 0, 1))
        self.layers.append(non_bottleneck_1d(32, 0, 1))

        self.layers.append(UpsamplerBlock(32,16))
        self.layers.append(non_bottleneck_1d(16, 0, 1))
        self.layers.append(non_bottleneck_1d(16, 0, 1))

        self.output_conv = nn.ConvTranspose2d( 16, num_classes, 2, stride=2, padding=0, output_padding=0, bias=True)

    def forward(self, input):
        output = input

        for layer in self.layers:
            output = layer(output)

        output = self.output_conv(output)

        return output

#ERFNet
class ERFNET_PRUNED(nn.Module):
    def __init__(self, num_classes, encoder=None):  #use encoder to pass pretrained encoder
        super().__init__()

        if (encoder == None):
            self.encoder = Encoder(num_classes)
        else:
            self.encoder = encoder
        self.decoder = Decoder(num_classes)

    def forward(self, input, only_encode=False):
        if only_encode:
            return self.encoder.forward(input, predict=True)
        else:
            output = self.encoder(input)    #predict=False by default
            return self.decoder.forward(output)


In [ ]:
erfnet_dec_pruned = ERFNET_PRUNED(20)

In [ ]:
from torchsummary import summary

summary(erfnet_dec_pruned, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 13, 112, 112]             364
         MaxPool2d-2          [-1, 3, 112, 112]               0
       BatchNorm2d-3         [-1, 16, 112, 112]              32
  DownsamplerBlock-4         [-1, 16, 112, 112]               0
            Conv2d-5           [-1, 48, 56, 56]           6,960
         MaxPool2d-6           [-1, 16, 56, 56]               0
       BatchNorm2d-7           [-1, 64, 56, 56]             128
  DownsamplerBlock-8           [-1, 64, 56, 56]               0
            Conv2d-9           [-1, 64, 56, 56]          12,352
           Conv2d-10           [-1, 64, 56, 56]          12,352
      BatchNorm2d-11           [-1, 64, 56, 56]             128
           Conv2d-12           [-1, 64, 56, 56]          12,352
           Conv2d-13           [-1, 64, 56, 56]          12,352
      BatchNorm2d-14           [-1, 64,

In [ ]:
import torch
import time
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import FakeData
import numpy as np

def inference_time(model, dataloader, device):
    model.eval()
    total_time = 0.0
    num_samples = 0

    with torch.no_grad():
        for inputs in dataloader:
            image = inputs[0]

            start_time = time.time()
            _ = model(image)  # Perform inference, adjust as needed based on your model's forward pass
            end_time = time.time()

            total_time += end_time - start_time
            num_samples += 1

    avg_time = total_time / num_samples
    return avg_time

# Set a random seed for reproducibility
torch.manual_seed(42)

device = torch.device("cpu")

# Create a data loader with random images
transform = transforms.Compose([transforms.ToTensor()])
fake_dataset = FakeData(transform=transform)
test_loader = DataLoader(fake_dataset, batch_size=1, shuffle=True, num_workers=2)

# Number of runs for obtaining average and standard deviation
num_runs = 2
inference_times = []

for run in range(num_runs):
    # Measure inference time for model1
    time_model1 = inference_time(erfnet_dec_pruned, test_loader, device)
    inference_times.append(time_model1)
    print(f"Run {run + 1}: Inference time for decoder pruned model: {time_model1:.6f} seconds per sample")

# Calculate average and standard deviation
avg_time_model1 = np.mean(inference_times)
std_dev_model1 = np.std(inference_times)

print(f"\nAverage inference time for decoder pruned model: {avg_time_model1:.6f} seconds per sample")
print(f"Standard deviation for decoder pruned model: {std_dev_model1:.6f} seconds per sample\n")


Run 1: Inference time for decoder pruned model: 0.170344 seconds per sample
Run 2: Inference time for decoder pruned model: 0.152544 seconds per sample

Average inference time for decoder pruned model: 0.161444 seconds per sample
Standard deviation for decoder pruned model: 0.008900 seconds per sample



In [ ]:
inference_times = []
erfnet_full = ERFNet(20)
erfnet_pruned = ERFNET_PRUNEDV2(20)

for run in range(num_runs):
    # Measure inference time for model1
    time_model1 = inference_time(erfnet_full, test_loader, device)
    inference_times.append(time_model1)
    print(f"Run {run + 1}: Inference time for full model: {time_model1:.6f} seconds per sample")

# Calculate average and standard deviation
avg_time_model1 = np.mean(inference_times)
std_dev_model1 = np.std(inference_times)

print(f"\nAverage inference time for full model: {avg_time_model1:.6f} seconds per sample")
print(f"Standard deviation for full model: {std_dev_model1:.6f} seconds per sample\n")

# Repeat the process for model2
inference_times = []

for run in range(num_runs):
    time_model2 = inference_time(erfnet_pruned, test_loader, device)
    inference_times.append(time_model2)
    print(f"Run {run + 1}: Inference time for pruned model: {time_model2:.6f} seconds per sample")

# Calculate average and standard deviation
avg_time_model2 = np.mean(inference_times)
std_dev_model2 = np.std(inference_times)

print(f"\nAverage inference time for pruned model: {avg_time_model2:.6f} seconds per sample")
print(f"Standard deviation for pruned model: {std_dev_model2:.6f} seconds per sample\n")

Run 1: Inference time for full model: 0.169797 seconds per sample
Run 2: Inference time for full model: 0.168253 seconds per sample

Average inference time for full model: 0.169025 seconds per sample
Standard deviation for full model: 0.000772 seconds per sample

Run 1: Inference time for pruned model: 0.139148 seconds per sample
Run 2: Inference time for pruned model: 0.137564 seconds per sample

Average inference time for pruned model: 0.138356 seconds per sample
Standard deviation for pruned model: 0.000792 seconds per sample



In [ ]:
macs, params = profile(erfnet_dec_pruned, inputs=(input, ))
macs, params = clever_format([macs, params], "%.3f")
macs, params

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.ConvTranspose2d'>.


('11.356G', '1.948M')

In [ ]:
macs, params = profile(model_quantized.eval(), inputs=(input, ))
macs, params = clever_format([macs, params], "%.3f")
macs, params ###AS IS POSSIBLE TO SEE, IT DOESNT RECOGNIZE  QUANTIZED MODULE

[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.


('152.640M', '0.000B')

In [ ]:
!pip install deepspeed

In [ ]:
from deepspeed.profiling.flops_profiler import get_model_profile
from deepspeed.accelerator import get_accelerator
from deepspeed.profiling.flops_profiler import FlopsProfiler

[2024-02-18 17:09:14,794] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
!pip install intel-extension-for-deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.1/120.1 kB 1.0 MB/s eta 0:00:00


In [ ]:
from deepspeed.profiling.flops_profiler import get_model_profile
from deepspeed.accelerator import get_accelerator
import intel_extension_for_pytorch
### DEEPSPEED DOESNT WORK ON CPU WITH THIS ENVIRONMENT
get_accelerator().device_name()

ERROR! Intel® Extension for PyTorch* needs to work with PyTorch 2.2.*, but PyTorch 2.1.0+cu121 is found. Please switch to the matching version and run again.


NameError: name 'exit' is not defined

In [ ]:
with get_accelerator().device():
    batch_size = 1
    flops, macs, params = get_model_profile(model=erfnet, # model
                                    input_shape=(batch_size, 3, 480, 480), # input shape to the model. If specified, the model takes a tensor with this shape as the only positional argument.
                                    args=None, # list of positional arguments to the model.
                                    kwargs=None, # dictionary of keyword arguments to the model.
                                    print_profile=True, # prints the model graph with the measured profile attached to each module
                                    detailed=True, # print the detailed profile
                                    module_depth=-1, # depth into the nested modules, with -1 being the inner most modules
                                    top_modules=1, # the number of top modules to print aggregated profile
                                    warm_up=10, # the number of warm-ups before measuring the time of each module
                                    as_string=True, # print raw numbers (e.g. 1000) or as human-readable strings (e.g. 1k)
                                    output_file=None, # path to the output file. If None, the profiler prints to stdout.
                                    ignore_modules=None) # the list of modules to ignore in the profiling

[2024-02-18 17:24:58,166] [INFO] [profiler.py:1205:get_model_profile] Flops profiler warming-up...
[2024-02-18 17:25:04,193] [INFO] [profiler.py:80:start_profile] Flops profiler started

-------------------------- DeepSpeed Flops Profiler --------------------------
Profile Summary at step 10:
Notations:
data parallel size (dp_size), model parallel size(mp_size),
number of parameters (params), number of multiply-accumulate operations(MACs),
number of floating-point operations (flops), floating-point operations per second (FLOPS),
fwd latency (forward propagation latency), bwd latency (backward propagation latency),
step (weights update latency), iter latency (sum of fwd, bwd and step latency)

params per GPU:                                                         2.07 M  
params of model = params per GPU * mp_size:                             0       
fwd MACs per GPU:                                                       11.7 GMACs
fwd flops per GPU:                                   

In [ ]:
with get_accelerator().device():
    batch_size = 1
    flops, macs, params = get_model_profile(model=model, # model
                                    input_shape=(batch_size, 3, 480, 480), # input shape to the model. If specified, the model takes a tensor with this shape as the only positional argument.
                                    args=None, # list of positional arguments to the model.
                                    kwargs=None, # dictionary of keyword arguments to the model.
                                    print_profile=True, # prints the model graph with the measured profile attached to each module
                                    detailed=True, # print the detailed profile
                                    module_depth=-1, # depth into the nested modules, with -1 being the inner most modules
                                    top_modules=1, # the number of top modules to print aggregated profile
                                    warm_up=10, # the number of warm-ups before measuring the time of each module
                                    as_string=True, # print raw numbers (e.g. 1000) or as human-readable strings (e.g. 1k)
                                    output_file=None, # path to the output file. If None, the profiler prints to stdout.
                                    ignore_modules=None) # the list of modules to ignore in the profiling

[2024-02-18 17:23:58,943] [INFO] [profiler.py:1205:get_model_profile] Flops profiler warming-up...
[2024-02-18 17:24:04,975] [INFO] [profiler.py:80:start_profile] Flops profiler started

-------------------------- DeepSpeed Flops Profiler --------------------------
Profile Summary at step 10:
Notations:
data parallel size (dp_size), model parallel size(mp_size),
number of parameters (params), number of multiply-accumulate operations(MACs),
number of floating-point operations (flops), floating-point operations per second (FLOPS),
fwd latency (forward propagation latency), bwd latency (backward propagation latency),
step (weights update latency), iter latency (sum of fwd, bwd and step latency)

params per GPU:                                                         13.43 M 
params of model = params per GPU * mp_size:                             0       
fwd MACs per GPU:                                                       13.34 GMACs
fwd flops per GPU:                                  

In [ ]:

with get_accelerator().device():
    batch_size = 1
    flops, macs, params = get_model_profile(model=model_quantized, # model
                                    input_shape=(batch_size, 3, 224, 224), # input shape to the model. If specified, the model takes a tensor with this shape as the only positional argument.
                                    args=None, # list of positional arguments to the model.
                                    kwargs=None, # dictionary of keyword arguments to the model.
                                    print_profile=True, # prints the model graph with the measured profile attached to each module
                                    detailed=True, # print the detailed profile
                                    module_depth=-1, # depth into the nested modules, with -1 being the inner most modules
                                    top_modules=1, # the number of top modules to print aggregated profile
                                    warm_up=10, # the number of warm-ups before measuring the time of each module
                                    as_string=True, # print raw numbers (e.g. 1000) or as human-readable strings (e.g. 1k)
                                    output_file=None, # path to the output file. If None, the profiler prints to stdout.
                                    ignore_modules=None) # the list of modules to ignore in the profiling

[2024-02-18 17:13:50,350] [INFO] [profiler.py:1205:get_model_profile] Flops profiler warming-up...
[2024-02-18 17:13:51,957] [INFO] [profiler.py:80:start_profile] Flops profiler started

-------------------------- DeepSpeed Flops Profiler --------------------------
Profile Summary at step 10:
Notations:
data parallel size (dp_size), model parallel size(mp_size),
number of parameters (params), number of multiply-accumulate operations(MACs),
number of floating-point operations (flops), floating-point operations per second (FLOPS),
fwd latency (forward propagation latency), bwd latency (backward propagation latency),
step (weights update latency), iter latency (sum of fwd, bwd and step latency)

params per GPU:                                                         0       
params of model = params per GPU * mp_size:                             0       
fwd MACs per GPU:                                                       0 MACs  
fwd flops per GPU:                                     

In [ ]:
prof = FlopsProfiler(model_quantized)
profile_step = 1
print_profile= True

prof.start_profile()
_ = model_quantized(torch.randn(1,3,480,480))
prof.stop_profile()
flops = prof.get_total_flops()
macs = prof.get_total_macs()
params = prof.get_total_params()
if print_profile:
  prof.print_model_profile(profile_step=profile_step)
prof.end_profile()

[2024-02-18 17:09:23,336] [INFO] [profiler.py:80:start_profile] Flops profiler started

-------------------------- DeepSpeed Flops Profiler --------------------------
Profile Summary at step 1:
Notations:
data parallel size (dp_size), model parallel size(mp_size),
number of parameters (params), number of multiply-accumulate operations(MACs),
number of floating-point operations (flops), floating-point operations per second (FLOPS),
fwd latency (forward propagation latency), bwd latency (backward propagation latency),
step (weights update latency), iter latency (sum of fwd, bwd and step latency)

params per GPU:                                                         0       
params of model = params per GPU * mp_size:                             0       
fwd MACs per GPU:                                                       0 MACs  
fwd flops per GPU:                                                      5.41 M  
fwd flops of model = fwd flops per GPU * mp_size:                       5.

In [ ]:
!ds_report


[2024-02-18 17:11:13,551] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables

In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity

with torch.profiler.profile(
    schedule=torch.profiler.schedule(
        wait=5, # During this phase profiler is not active.
        warmup=2, # During this phase profiler starts tracing, but the results are discarded.
        active=6, # During this phase profiler traces and records data.
        repeat=2), # Specifies an upper bound on the number of cycles.
    with_stack=True # Enable stack tracing, adds extra profiling overhead.
) as profiler:
      _= model_quantized(torch.randn(1,3,480,480))
      profiler.step() # Send the signal to the profiler that the next step has started.

In [ ]:
with profile(activities=[
        ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_forward"):
        model_quantized(torch.randn(1,3,480,480))

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                    model_forward         2.69%       7.527ms       100.00%     279.499ms     279.499ms             1  
                      aten::randn         0.12%     328.000us         2.06%       5.758ms       5.758ms             1  
                      aten::empty         0.11%     303.000us         0.11%     303.000us       6.184us            49  
                    aten::normal_         1.93%       5.400ms         1.93%       5.400ms       5.400ms             1  
        aten::quantize_per_tensor         0.19%     529.000us         0.19%     541.000us      77.286us             7  
                       aten::item       